In [1]:
import pandas as pd
import numpy as np
import os
import sys
import json
import requests
import gc
from tqdm import tqdm

from google.cloud import bigquery
from google.cloud.bigquery import job
from datetime import date, timedelta

PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)

#  클라이언트 설정



# bycommerce 쿼리

### rfm-trg1-create-byshop-1213

In [2]:
bycommerce_1 =  """CREATE or Replace TABLE ballosodeuk.ynam.rfm_table_bycommerce AS (
WITH date_vars AS (
    SELECT date({end_date}) as standard_date  -- B: 기준일
)

,order_complete as (
    select event_dt, event_dttm, user_id, airbridge_device_id, platform,transaction_id, event_category, event_label, event_action, price as event_value
    from `ballosodeuk.dw.fact_airbridge_event_order`
    where 
        Date(event_dt) between date("2024-03-01") and date({end_date})
        and event_action not like "^PAY" 
        and event_label != "쇼핑"
        and user_id not like 'IU_%' 
        and user_id is not Null
),

shortcut_order as (
select event_dt, user_id, min(airbridge_device_id) as airbridge_device_id, transaction_id, min(platform) as platform, event_label, event_action, min(event_category) as event_category, sum(Total_Revenue) as Total_Revenue
from( 
    select 
    event_dt, event_dttm, user_id, airbridge_device_id, platform,transaction_id, event_category, event_action, 
    CASE 
        WHEN event_action in ('balso1sa1','balso1sr1','balso2sr1','balso2sa2') then '바로가기'
        WHEN event_action = 'balso1sr2' then '퀴즈쿠팡'
        WHEN event_action = 'balso2sa1' then '챌린지인증쿠팡'
    END AS event_label, 
    round(sum(event_value) * 0.046) as Total_Revenue
    from order_complete
    where event_category = "Order Complete (App)"
    GROUP BY event_dt, event_dttm, user_id, airbridge_device_id,transaction_id, platform, event_category, event_label, event_action
)
group by event_dt, user_id, transaction_id, event_label, event_action
),

shortcut_refund as (
select event_dt, user_id, min(airbridge_device_id) as airbridge_device_id, transaction_id, min(platform) as platform, event_label, event_action, min(event_category) as event_category, sum(Total_Revenue) as Total_Revenue
from( 
    select 
    event_dt, event_dttm, user_id, airbridge_device_id, platform,transaction_id, event_category, event_action, 
    CASE 
        WHEN event_action in ('balso1sa1','balso1sr1','balso2sr1','balso2sa2') then '바로가기'
        WHEN event_action = 'balso1sr2' then '퀴즈쿠팡'
        WHEN event_action = 'balso2sa1' then '챌린지인증쿠팡'
    END AS event_label, 
    round(sum(event_value) * 0.046) * -1 as Total_Revenue
    from order_complete
    where event_category = "Order Cancel (App)"
    GROUP BY event_dt, event_dttm, user_id, airbridge_device_id,transaction_id, platform, event_category, event_label, event_action
)
group by event_dt, user_id, transaction_id, event_label, event_action
),

shortcut as (
select o.event_dt, o.user_id, o.airbridge_device_id, o.transaction_id, o.platform, o.event_label, o.event_action, o.event_category, 
case when o.Total_Revenue + COALESCE(r.Total_Revenue, 0) < 0 then 0 else o.Total_Revenue + COALESCE(r.Total_Revenue, 0) end as Total_Revenue
    -- 구매 <> 환불의 금액이 100% 매칭이 안될 수 있음. 해당 케이스는 0원으로 처리
from shortcut_order o
left join shortcut_refund r on o.user_id = r.user_id and o.transaction_id = r.transaction_id
),

dynamic_order AS (
    SELECT 
    cast(FORMAT_DATE('%Y-%m-%d', DATETIME(date)) as Date) as event_dt, 
    subParam AS user_id, 
    'Order Complete (App)' AS event_category, 
    'dynamic' AS event_action, 
    'coin' As event_label,
    SUM(commission) AS Total_Revenue
    FROM `ballosodeuk.external_mart.cpDynamic_orders` 
    WHERE 
    date between date("2024-03-01") and date_sub(current_date(),interval 1 day)
    AND subParam IS NOT NULL
    GROUP BY 
    event_dt, 
    user_id, 
    event_category, 
    event_action
),

dynamic_refund as (
SELECT 
    FORMAT_DATE('%Y-%m-%d', PARSE_DATE('%Y%m%d', date)) as event_dt, 
    orderDate as order_date, 
    subParam As user_id, 
    'Order Cancel (App)' As event_category,
    'dynamic' AS event_action, 
    'coin' As event_label,
    SUM(commission) AS Total_Revenue
FROM `ballosodeuk.external_mart.cpDynamic_cancels` 
WHERE 
    parse_date('%Y%m%d',date) BETWEEN date("2024-03-01") and date_sub(current_date(),interval 1 day)
    and subParam Is Not Null
group by 
    event_dt, order_date, user_id, event_category, event_action  
),

dynamic as (
select o.event_dt, o.user_id, o.event_category, o.event_action, o.event_label, 
case when o.Total_Revenue + COALESCE(r.Total_Revenue, 0) < 0 then 0 else o.Total_Revenue + COALESCE(r.Total_Revenue, 0) end as Total_Revenue
    -- 구매 <> 환불의 금액이 100% 매칭이 안될 수 있음. 해당 케이스는 0원으로 처리
from dynamic_order o
left join dynamic_refund r on o.user_id = r.user_id and o.event_dt = r.order_date
),

combined_commerce AS (
SELECT event_dt, user_id, event_category, event_label, Total_Revenue
FROM shortcut
UNION ALL
SELECT event_dt, user_id, event_category, event_label, Total_Revenue
FROM dynamic
)

,user_properties_filled as (
select *
from ballosodeuk.dw.dim_airbridge_member
)

,intermediate_commerce_data AS (
SELECT 
    up.user_id,
    up.platform,
    up.join_dt,
    DATE_DIFF(DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY), DATE(up.join_dt), DAY) AS cum_lifetime,
    max(up.total_accumulate_cash) as total_accumulate_cash,
    max(up.current_cash) as current_cash,
    max(cc.event_dt) as event_dt,
    COUNT(DISTINCT cc.event_dt) as ByCommerce_F,
    SUM(COALESCE(cc.Total_Revenue, 0)) as ByCommerce_M
FROM user_properties_filled up
LEFT JOIN combined_commerce cc ON up.user_id = cc.user_id
WHERE 
    cc.event_dt IS NULL OR 
    DATE(cc.event_dt) <= date({end_date})
GROUP BY 
    up.user_id, up.platform, up.join_dt
)

,commerce_data AS (
SELECT 
    user_id,
    platform,
    join_dt,
    cum_lifetime,
    total_accumulate_cash,
    current_cash,
    CASE 
        WHEN max(event_dt) IS NULL THEN NULL
        ELSE DATE_DIFF(date({end_date}), max(event_dt), DAY)
    END as recency,
    ByCommerce_F as frequency,
    ByCommerce_M as monetary
FROM intermediate_commerce_data
GROUP BY 
    user_id, platform, join_dt, cum_lifetime,
    total_accumulate_cash, current_cash, ByCommerce_F, ByCommerce_M
)

SELECT *, date({end_date}) as snapshot_dt
FROM commerce_data
WHERE monetary > 0
)

"""


### rfm-trg2-create-byshop-categoryPower-1223

In [3]:
bycommerce_2 = """CREATE or Replace TABLE ballosodeuk.ynam.rfm_table_byshop_category_power AS (

WITH order_complete AS (
    SELECT event_dt, event_dttm, user_id, airbridge_device_id, platform,
        transaction_id, event_category, event_label, event_action, name,
        price AS event_value
    FROM `ballosodeuk.dw.fact_airbridge_event_order`
    WHERE 
        DATE(event_dt) BETWEEN date("2024-03-01") AND date({end_date})
        AND event_action NOT LIKE "^PAY"
        AND event_label != "쇼핑"
        AND user_id NOT LIKE 'IU_%'
        AND user_id IS NOT NULL
),

shortcut_order AS (
SELECT 
    event_dt,
    user_id,
    MIN(airbridge_device_id) AS airbridge_device_id,
    transaction_id,
    MIN(platform) AS platform,
    event_label,
    event_action,
    MIN(name) AS name,
    MIN(event_category) AS event_category,
    SUM(Total_Revenue) AS Total_Revenue
FROM (
    SELECT 
    event_dt, event_dttm, user_id, airbridge_device_id, platform,
    transaction_id, event_category, event_action, name,
    CASE 
        WHEN event_action IN ('balso1sa1','balso1sr1','balso2sr1','balso2sa2') THEN '바로가기'
        WHEN event_action = 'balso1sr2' THEN '퀴즈쿠팡'
        WHEN event_action = 'balso2sa1' THEN '챌린지인증쿠팡'
    END AS event_label,
    ROUND(SUM(event_value) * 0.046) AS Total_Revenue
    FROM order_complete
    WHERE event_category = "Order Complete (App)"
    GROUP BY event_dt, event_dttm, user_id, airbridge_device_id, platform,
            transaction_id, event_category, event_label, event_action, name
)
GROUP BY event_dt, user_id, transaction_id, event_label, event_action
),

shortcut_refund AS (
SELECT 
    event_dt,
    user_id,
    MIN(airbridge_device_id) AS airbridge_device_id,
    transaction_id,
    MIN(platform) AS platform,
    event_label,
    event_action,
    MIN(event_category) AS event_category,
    SUM(Total_Revenue) AS Total_Revenue
FROM (
    SELECT 
    event_dt, event_dttm, user_id, airbridge_device_id, platform, transaction_id,
    event_category, event_action, 
    CASE 
        WHEN event_action IN ('balso1sa1','balso1sr1','balso2sr1','balso2sa2') THEN '바로가기'
        WHEN event_action = 'balso1sr2' THEN '퀴즈쿠팡'
        WHEN event_action = 'balso2sa1' THEN '챌린지인증쿠팡'
    END AS event_label,
    ROUND(SUM(event_value) * 0.046) * -1 AS Total_Revenue
    FROM order_complete
    WHERE event_category = "Order Cancel (App)"
    GROUP BY event_dt, event_dttm, user_id, airbridge_device_id, platform,
            transaction_id, event_category, event_label, event_action
)
GROUP BY event_dt, user_id, transaction_id, event_label, event_action
),

shortcut AS (
SELECT 
    o.event_dt,
    o.user_id,
    o.airbridge_device_id,
    o.transaction_id,
    o.platform,
    o.name,
    o.event_label,
    o.event_action,
    o.event_category,
    CASE WHEN o.Total_Revenue + COALESCE(r.Total_Revenue, 0) < 0
        THEN 0
        ELSE o.Total_Revenue + COALESCE(r.Total_Revenue, 0)
    END AS Total_Revenue
FROM shortcut_order o
LEFT JOIN shortcut_refund r
    ON o.user_id = r.user_id
AND o.transaction_id = r.transaction_id
),

dynamic_order AS (
    SELECT 
    CAST(FORMAT_DATE('%Y-%m-%d', DATETIME(date)) AS DATE) AS event_dt, 
    subParam AS user_id, 
    'Order Complete (App)' AS event_category, 
    'dynamic' AS event_action, 
    'coin' AS event_label,
    MIN(productName) AS name,
    SUM(commission) AS Total_Revenue
    FROM `ballosodeuk.external_mart.cpDynamic_orders`
    WHERE 
    date BETWEEN date("2024-03-01") AND date({end_date})
    AND subParam IS NOT NULL
    GROUP BY event_dt, user_id, event_category, event_action
),

dynamic_refund AS (
SELECT 
    FORMAT_DATE('%Y-%m-%d', PARSE_DATE('%Y%m%d', date)) AS event_dt,
    orderDate AS order_date,
    subParam AS user_id, 
    'Order Cancel (App)' AS event_category,
    'dynamic' AS event_action, 
    'coin' AS event_label,
    SUM(commission) AS Total_Revenue
FROM `ballosodeuk.external_mart.cpDynamic_cancels`
WHERE 
    PARSE_DATE('%Y%m%d', date) BETWEEN date("2024-03-01") AND date({end_date})
    AND subParam IS NOT NULL
GROUP BY event_dt, order_date, user_id, event_category, event_action
),

dynamic AS (
SELECT 
    o.event_dt,
    o.user_id,
    o.event_category,
    o.event_action,
    o.event_label,
    o.name,
    CASE WHEN o.Total_Revenue + COALESCE(r.Total_Revenue, 0) < 0
        THEN 0
        ELSE o.Total_Revenue + COALESCE(r.Total_Revenue, 0)
    END AS Total_Revenue
FROM dynamic_order o
LEFT JOIN dynamic_refund r
    ON o.user_id = r.user_id
AND o.event_dt = r.order_date
),

combined_commerce AS (
SELECT event_dt, user_id, event_category, event_label, name, Total_Revenue
    FROM shortcut
UNION ALL
SELECT event_dt, user_id, event_category, event_label, name, Total_Revenue
    FROM dynamic
),

/* ------ 상품명 vs 카테고리 매핑 ------ */
category_raw AS (
SELECT 
    c.*,
    k.category1,
    k.category2
FROM combined_commerce c
LEFT JOIN ballosodeuk.external_mart.cpProduct_keyword k
    ON k.name = c.name
),

/* ------ depth 카테고리 정제 ------ */
depth1_raw AS (
SELECT 
    c.*,
    REPLACE(category1, '/', '-') AS depth1
FROM category_raw c
),
depth2_raw AS (
SELECT
    c.*,
    REPLACE(category2, '/', '-') AS depth2,
    REPLACE(category1, '/', '-') AS depth1
FROM category_raw c
),

-------------------------------------------------------------------------------
-- [B-1] Depth1만 처리 (Top3)
-------------------------------------------------------------------------------
user_category_stats_dept1 AS (
SELECT 
    user_id,
    depth1,
    DATE_DIFF(MAX(event_dt), CURRENT_DATE(), DAY) AS latest_order_dt,
    COUNT(DISTINCT event_dt) AS order_count
FROM depth1_raw
WHERE category1 IS NOT NULL
    AND TRIM(category1) != ''
    AND TRIM(category1) != 'None'
GROUP BY user_id, depth1
),

raw_weight_cte_depth1 AS (
SELECT
    user_id,
    depth1,
    ABS(latest_order_dt) AS days,
    order_count,
    SUM(order_count) OVER (PARTITION BY user_id) AS total_order_count
FROM user_category_stats_dept1
),

score_cte_depth1 AS (
SELECT
    user_id,
    depth1,
    days,
    order_count,
    SAFE_DIVIDE(order_count, total_order_count) AS freq_weight,
    EXP(-0.1 * days) AS recency_weight,
    0.6 * SAFE_DIVIDE(order_count, total_order_count)
    + 0.4 * EXP(-0.1 * days) AS final_score
FROM raw_weight_cte_depth1
),

final_calc_depth1 AS (
SELECT
    user_id,
    depth1,
    final_score,
    SUM(final_score) OVER (PARTITION BY user_id) AS sum_score
FROM score_cte_depth1
),

result_depth1 AS (
SELECT
    user_id,
    depth1,
    final_score,
    ROUND(SAFE_DIVIDE(final_score, sum_score), 2) AS interest_ratio
FROM final_calc_depth1
),

rank_table_depth1 AS (
SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY interest_ratio DESC) AS ranking
FROM result_depth1
),

pivot_table_depth1 AS (
SELECT 
    user_id,
    MAX(CASE WHEN ranking = 1 THEN depth1 END) AS ranking_1_1,
    MAX(CASE WHEN ranking = 2 THEN depth1 END) AS ranking_1_2,
    MAX(CASE WHEN ranking = 3 THEN depth1 END) AS ranking_1_3,

    MAX(CASE WHEN ranking = 1 THEN interest_ratio END) AS power_1_1,
    MAX(CASE WHEN ranking = 2 THEN interest_ratio END) AS power_1_2,
    MAX(CASE WHEN ranking = 3 THEN interest_ratio END) AS power_1_3
FROM rank_table_depth1
WHERE ranking <= 3
GROUP BY user_id
),

-------------------------------------------------------------------------------
-- [B-2] Depth1(Depth2) 처리 (Top6)
-------------------------------------------------------------------------------
-- 1) depth2가 NULL이면 '없음'으로 치환 → depth1( depth2 ) 문자열로 합치기
user_category_stats_dept1plus AS (
SELECT 
    user_id,
    CONCAT(
    depth1,
    '(',
    COALESCE(NULLIF(TRIM(depth2), ''), '없음'),
    ')'
    ) AS depth1plus,
    DATE_DIFF(MAX(event_dt), CURRENT_DATE(), DAY) AS latest_order_dt,
    COUNT(DISTINCT event_dt) AS order_count
FROM depth2_raw
WHERE category1 IS NOT NULL
    AND TRIM(category1) != ''
    AND TRIM(category1) != 'None'
    -- depth2가 비어도 상관없음, 위에서 COALESCE( '없음' ) 처리
GROUP BY user_id, depth1, depth2
),

raw_weight_cte_depth1plus AS (
SELECT
    user_id,
    depth1plus,
    ABS(latest_order_dt) AS days,
    order_count,
    SUM(order_count) OVER (PARTITION BY user_id) AS total_order_count
FROM user_category_stats_dept1plus
),

score_cte_depth1plus AS (
SELECT
    user_id,
    depth1plus,
    days,
    order_count,
    SAFE_DIVIDE(order_count, total_order_count) AS freq_weight,
    EXP(-0.1 * days) AS recency_weight,
    0.6 * SAFE_DIVIDE(order_count, total_order_count)
    + 0.4 * EXP(-0.1 * days) AS final_score
FROM raw_weight_cte_depth1plus
),

final_calc_depth1plus AS (
SELECT
    user_id,
    depth1plus,
    final_score,
    SUM(final_score) OVER (PARTITION BY user_id) AS sum_score
FROM score_cte_depth1plus
),

result_depth1plus AS (
SELECT
    user_id,
    depth1plus,
    final_score,
    ROUND(SAFE_DIVIDE(final_score, sum_score), 2) AS interest_ratio
FROM final_calc_depth1plus
),

rank_table_depth1plus AS (
SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY interest_ratio DESC) AS ranking
FROM result_depth1plus
),

-- 2) Top6 Pivot
pivot_table_depth1plus AS (
SELECT 
    user_id,

    MAX(CASE WHEN ranking=1 THEN depth1plus END) AS ranking_2_1,
    MAX(CASE WHEN ranking=2 THEN depth1plus END) AS ranking_2_2,
    MAX(CASE WHEN ranking=3 THEN depth1plus END) AS ranking_2_3,
    MAX(CASE WHEN ranking=4 THEN depth1plus END) AS ranking_2_4,
    MAX(CASE WHEN ranking=5 THEN depth1plus END) AS ranking_2_5,
    MAX(CASE WHEN ranking=6 THEN depth1plus END) AS ranking_2_6,

    MAX(CASE WHEN ranking=1 THEN interest_ratio END) AS power_2_1,
    MAX(CASE WHEN ranking=2 THEN interest_ratio END) AS power_2_2,
    MAX(CASE WHEN ranking=3 THEN interest_ratio END) AS power_2_3,
    MAX(CASE WHEN ranking=4 THEN interest_ratio END) AS power_2_4,
    MAX(CASE WHEN ranking=5 THEN interest_ratio END) AS power_2_5,
    MAX(CASE WHEN ranking=6 THEN interest_ratio END) AS power_2_6

FROM rank_table_depth1plus
WHERE ranking <= 6
GROUP BY user_id
),

-------------------------------------------------------------------------------
-- [C] 최종 JOIN: depth1 전용 Top3 + depth1(depth2) Top6
-------------------------------------------------------------------------------
final_join AS (
SELECT 
    p1.user_id,

    -- depth1 전용
    p1.ranking_1_1, p1.ranking_1_2, p1.ranking_1_3,
    p1.power_1_1,   p1.power_1_2,   p1.power_1_3,

    -- depth1(depth2) 전용 (Top6)
    p2.ranking_2_1, p2.ranking_2_2, p2.ranking_2_3,
    p2.ranking_2_4, p2.ranking_2_5, p2.ranking_2_6,

    p2.power_2_1,   p2.power_2_2,   p2.power_2_3,
    p2.power_2_4,   p2.power_2_5,   p2.power_2_6

FROM pivot_table_depth1 p1
LEFT JOIN pivot_table_depth1plus p2
    ON p1.user_id = p2.user_id
)

SELECT *
FROM final_join
ORDER BY user_id


)
"""


###  rfm-trg3-create-byshop-churn-1-1231

In [4]:
bycommerce_3 = """CREATE OR REPLACE TABLE ballosodeuk.ynam.rfm_table_byshop_before_survive_prop AS (

WITH order_complete AS (
    SELECT 
        event_dt, 
        event_dttm, 
        user_id, 
        airbridge_device_id, 
        platform,
        transaction_id, 
        event_category, 
        event_label, 
        event_action, 
        price as event_value
    FROM `ballosodeuk.dw.fact_airbridge_event_order`
    WHERE 
        DATE(event_dt) BETWEEN date("2024-03-01") AND date({end_date})
        AND event_action NOT LIKE "^PAY" 
        AND event_label != "쇼핑"
        AND user_id NOT LIKE 'IU_%' 
        AND user_id IS NOT NULL
),

shortcut_order AS (
    SELECT 
        event_dt, 
        user_id, 
        MIN(airbridge_device_id) as airbridge_device_id, 
        transaction_id, 
        MIN(platform) as platform, 
        event_label, 
        event_action, 
        MIN(event_category) as event_category, 
        SUM(Total_Revenue) as Total_Revenue
    FROM( 
        SELECT 
            event_dt, 
            event_dttm, 
            user_id, 
            airbridge_device_id, 
            platform,
            transaction_id, 
            event_category, 
            event_action, 
            CASE 
                WHEN event_action IN ('balso1sa1','balso1sr1','balso2sr1','balso2sa2') THEN '바로가기'
                WHEN event_action = 'balso1sr2' THEN '퀴즈쿠팡'
                WHEN event_action = 'balso2sa1' THEN '챌린지인증쿠팡'
            END AS event_label, 
            ROUND(SUM(event_value) * 0.046) as Total_Revenue
        FROM order_complete
        WHERE event_category = "Order Complete (App)"
        GROUP BY 
            event_dt, 
            event_dttm, 
            user_id, 
            airbridge_device_id,
            transaction_id, 
            platform, 
            event_category, 
            event_label, 
            event_action
    )
    GROUP BY 
        event_dt, 
        user_id, 
        transaction_id, 
        event_label, 
        event_action
),

shortcut_refund AS (
    SELECT 
        event_dt, 
        user_id, 
        MIN(airbridge_device_id) as airbridge_device_id, 
        transaction_id, 
        MIN(platform) as platform, 
        event_label, 
        event_action, 
        MIN(event_category) as event_category, 
        SUM(Total_Revenue) as Total_Revenue
    FROM( 
        SELECT 
            event_dt, 
            event_dttm, 
            user_id, 
            airbridge_device_id, 
            platform,
            transaction_id, 
            event_category, 
            event_action, 
            CASE 
                WHEN event_action IN ('balso1sa1','balso1sr1','balso2sr1','balso2sa2') THEN '바로가기'
                WHEN event_action = 'balso1sr2' THEN '퀴즈쿠팡'
                WHEN event_action = 'balso2sa1' THEN '챌린지인증쿠팡'
            END AS event_label, 
            ROUND(SUM(event_value) * 0.046) * -1 as Total_Revenue
        FROM order_complete
        WHERE event_category = "Order Cancel (App)"
        GROUP BY 
            event_dt, 
            event_dttm, 
            user_id, 
            airbridge_device_id,
            transaction_id, 
            platform, 
            event_category, 
            event_label, 
            event_action
    )
    GROUP BY 
        event_dt, 
        user_id, 
        transaction_id, 
        event_label, 
        event_action
),

shortcut AS (
    SELECT 
        o.event_dt, 
        o.user_id, 
        o.airbridge_device_id, 
        o.transaction_id, 
        o.platform, 
        o.event_label, 
        o.event_action, 
        o.event_category, 
        o.Total_Revenue + COALESCE(r.Total_Revenue, 0) as Total_Revenue
    FROM shortcut_order o
    LEFT JOIN shortcut_refund r 
        ON o.user_id = r.user_id 
        AND o.transaction_id = r.transaction_id
),

dynamic_order AS (
    SELECT 
        CAST(FORMAT_DATE('%Y-%m-%d', DATETIME(date)) AS Date) as event_dt, 
        subParam AS user_id, 
        'Order Complete (App)' AS event_category, 
        'dynamic' AS event_action, 
        'coin' As event_label,
        SUM(commission) AS Total_Revenue
    FROM `ballosodeuk.external_mart.cpDynamic_orders` 
    WHERE 
        date BETWEEN date("2024-06-19") AND date({end_date})
        AND subParam IS NOT NULL
    GROUP BY 
        event_dt, 
        user_id, 
        event_category, 
        event_action
),

dynamic_refund AS (
    SELECT 
        FORMAT_DATE('%Y-%m-%d', PARSE_DATE('%Y%m%d', date)) as event_dt, 
        orderDate as order_date, 
        subParam As user_id, 
        'Order Cancel (App)' As event_category,
        'dynamic' AS event_action, 
        'coin' As event_label,
        SUM(commission) AS Total_Revenue
    FROM `ballosodeuk.external_mart.cpDynamic_cancels` 
    WHERE 
        PARSE_DATE('%Y%m%d', date) BETWEEN "2024-06-19" AND date({end_date})
        AND subParam IS NOT NULL
    GROUP BY 
        event_dt, 
        order_date, 
        user_id, 
        event_category, 
        event_action
),

dynamic AS (
    SELECT 
        o.event_dt, 
        o.user_id, 
        o.event_category, 
        o.event_action, 
        o.event_label, 
        o.Total_Revenue + COALESCE(r.Total_Revenue, 0) as Total_Revenue
    FROM dynamic_order o
    LEFT JOIN dynamic_refund r 
        ON o.user_id = r.user_id 
        AND o.event_dt = r.order_date
),

combined_commerce AS (
    SELECT event_dt, user_id, event_category, event_label, Total_Revenue
    FROM shortcut
    UNION ALL
    SELECT event_dt, user_id, event_category, event_label, Total_Revenue
    FROM dynamic
)

select * from combined_commerce
)
"""


### rfm-trg3-create-byshop-churn-2-0102

In [5]:
bycommerce_4 = """create or replace table ballosodeuk.ynam.rfm_table_byshop_survive_prop as (
with base_data AS (
    SELECT 
        user_id,
        event_dt as order_dt,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY event_dt DESC) as recency_rank,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY event_dt ASC) as purchase_rank
    FROM ballosodeuk.ynam.rfm_table_byshop_before_survive_prop
    GROUP BY user_id, event_dt
),

recent_purchase as (
    SELECT 
        user_id,
        order_dt as latest_order_dt,
        date_diff(date({end_date_plus}), order_dt, day) as days_since_last_purchase
    FROM base_data
    WHERE recency_rank = 1
),

purchase_intervals AS (
    SELECT 
        user_id,
        order_dt,
        purchase_rank,
        LEAD(order_dt) OVER (PARTITION BY user_id ORDER BY order_dt) as next_order_date,
        date_diff(
            LEAD(order_dt) OVER (PARTITION BY user_id ORDER BY order_dt),
            order_dt,
            day
        ) as days_between_orders
    FROM base_data
    WHERE purchase_rank <= 15
),

user_stats as (
    SELECT 
        user_id,
        stddev(days_between_orders) as cycle_stddev
    FROM purchase_intervals
    WHERE days_between_orders is not null
    GROUP BY user_id
),

current_trailing AS (
    SELECT 
        user_id,
        round(avg(days_between_orders), 1) as current_trailing_term,
        count(*) as current_count
    FROM purchase_intervals
    WHERE days_between_orders IS NOT NULL
    AND purchase_rank >= 1 
    AND purchase_rank <= 3
    GROUP BY user_id
),

prev_trailing AS (
    SELECT 
        user_id,
        round(avg(days_between_orders), 1) as prev_trailing_term,
        count(*) as prev_count
    FROM purchase_intervals
    WHERE days_between_orders IS NOT NULL
    AND purchase_rank >= 2
    AND purchase_rank <= 4
    GROUP BY user_id
),

survival_base AS (
    SELECT 
        c.user_id,
        r.days_since_last_purchase,
        c.current_trailing_term,
        p.prev_trailing_term,
        s.cycle_stddev,
        ROUND(((c.current_trailing_term - p.prev_trailing_term) / 
        NULLIF(p.prev_trailing_term, 0)) * 100, 1) as cycle_change_rate,
        ROUND((s.cycle_stddev / NULLIF(c.current_trailing_term, 0)) * 100, 1) as cycle_variation_rate
    FROM current_trailing c
    LEFT JOIN prev_trailing p ON c.user_id = p.user_id
    LEFT JOIN user_stats s ON c.user_id = s.user_id
    LEFT JOIN recent_purchase r ON c.user_id = r.user_id
),

term_stats AS (
    SELECT 
        current_trailing_term,
        LN(NULLIF(current_trailing_term, 0)) as log_term,
        STDDEV(LN(NULLIF(current_trailing_term, 0))) OVER () as log_stddev,
        AVG(LN(NULLIF(current_trailing_term, 0))) OVER () as log_mean,
        STDDEV(current_trailing_term) OVER () as pop_stddev,
        AVG(current_trailing_term) OVER () as pop_mean
    FROM survival_base
    WHERE current_trailing_term IS NOT NULL
    AND current_trailing_term > 0
),

median_stats AS ( 
    SELECT 
        APPROX_QUANTILES(current_trailing_term, 2)[OFFSET(1)] as median_term,
        APPROX_QUANTILES(LN(NULLIF(current_trailing_term, 0)), 2)[OFFSET(1)] as log_median_term
    FROM survival_base
    WHERE current_trailing_term IS NOT NULL
    AND current_trailing_term > 0
),

mad_stats AS (
    SELECT
        APPROX_QUANTILES(
            ABS(s.current_trailing_term - m.median_term), 
            2
        )[OFFSET(1)] as mad,
        APPROX_QUANTILES(
            ABS(LN(NULLIF(s.current_trailing_term, 0)) - m.log_median_term),
            2
        )[OFFSET(1)] as log_mad
    FROM survival_base s
    CROSS JOIN median_stats m
    WHERE s.current_trailing_term IS NOT NULL
    AND s.current_trailing_term > 0
),

robust_bounds AS (
    SELECT
        s.user_id,
        s.current_trailing_term,
        t.pop_mean,
        0.6745 * (LN(NULLIF(s.current_trailing_term, 0)) - m.log_median_term) / NULLIF(mad.log_mad, 0) as modified_zscore,
        s.days_since_last_purchase
    FROM survival_base s
    CROSS JOIN (SELECT DISTINCT pop_mean FROM term_stats) t
    CROSS JOIN median_stats m
    CROSS JOIN mad_stats mad
    WHERE s.current_trailing_term IS NOT NULL
    AND s.current_trailing_term > 0
),

churn_data AS (
    SELECT 
        r.user_id,
        r.current_trailing_term,
        CASE
            WHEN ABS(r.modified_zscore) > 3.5 THEN
                CASE WHEN r.days_since_last_purchase > r.pop_mean * 2 THEN 1 ELSE 0 END
            ELSE
                CASE WHEN r.days_since_last_purchase > r.current_trailing_term * 2 THEN 1 ELSE 0 END
        END as churn_flag
    FROM robust_bounds r
),

average_churn_rate AS (
    SELECT 
        SUM(churn_flag) / COUNT(*) AS avg_churn_rate,  
        AVG(current_trailing_term) as avg_group_term
    FROM churn_data
),

filled_time_points AS (
    SELECT DISTINCT time_point
    FROM UNNEST(GENERATE_ARRAY(
        0, 
        (SELECT MAX(FLOOR(days_since_last_purchase / 7) * 7) FROM survival_base),
        7
    )) as time_point
),

survival_base_aggregated AS (
    SELECT 
        FLOOR(s.days_since_last_purchase / 7) * 7 AS time_point,
        COUNT(*) AS n_risk,
        SUM(c.churn_flag) AS n_events
    FROM survival_base s
    LEFT JOIN churn_data c ON s.user_id = c.user_id
    GROUP BY FLOOR(s.days_since_last_purchase / 7) * 7
),

km_estimate AS (
    SELECT 
        time_point,
        n_risk,
        n_events,
        ROUND(
            CASE 
                WHEN n_risk > 0 THEN (1 - SAFE_DIVIDE(n_events, n_risk))
                ELSE 1 
            END, 
        4) as base_survival_prob,
        
        ROUND(
            CASE 
                WHEN n_risk > 0 THEN 
                    (1 - SAFE_DIVIDE(n_events, n_risk)) * 
                    (1 - COALESCE((SELECT avg_churn_rate FROM average_churn_rate), 0))
                ELSE 1 
            END,
        4) as weighted_survival_prob,
        
        ROUND(
            EXP(
                SUM(LN(
                    GREATEST(
                        CASE 
                            WHEN n_risk > 0 THEN (1 - SAFE_DIVIDE(n_events, n_risk))
                            ELSE 1
                        END,
                        0.0001
                    )
                )) OVER (ORDER BY time_point)
            ), 
        4) as cumulative_survival_prob
    FROM survival_base_aggregated
),

min_survival_prob AS (
    SELECT MIN(cumulative_survival_prob) as min_survival_prob
    FROM km_estimate
),

individual_survival AS (
    SELECT 
        s.user_id,
        s.days_since_last_purchase,
        s.current_trailing_term,
        s.prev_trailing_term,
        c.churn_flag,
        r.modified_zscore,
        acr.avg_churn_rate,
        acr.avg_group_term,
        CASE 
            WHEN ABS(r.modified_zscore) > 3.5 THEN 
                0.7 * EXP(-r.days_since_last_purchase / r.pop_mean) +
                0.3 * COALESCE(k.cumulative_survival_prob, m.min_survival_prob)
            ELSE
                0.7 * EXP(-r.days_since_last_purchase / NULLIF(s.current_trailing_term, 0)) +
                0.3 * COALESCE(k.cumulative_survival_prob, m.min_survival_prob)
        END as survival_prob
    FROM survival_base s
    LEFT JOIN churn_data c ON s.user_id = c.user_id
    LEFT JOIN robust_bounds r ON s.user_id = r.user_id
    CROSS JOIN average_churn_rate acr 
    LEFT JOIN km_estimate k 
        ON k.time_point = FLOOR(s.days_since_last_purchase / 7) * 7
    CROSS JOIN min_survival_prob m
),

final_analysis AS (
    SELECT 
        s.user_id,
        s.days_since_last_purchase,
        s.current_trailing_term,
        s.prev_trailing_term,
        round(s.cycle_stddev) as cycle_stddev,
        i.modified_zscore,
        i.avg_churn_rate as demographic_churn_rate,
        i.churn_flag,
        round(i.survival_prob,2) as survival_prob,
        CASE 
            WHEN ABS(i.modified_zscore) > 3.5 THEN 
                GREATEST(ROUND(-r.pop_mean * LN(0.5) - s.days_since_last_purchase),0)
            ELSE
                GREATEST(ROUND(-s.current_trailing_term * LN(0.5) - s.days_since_last_purchase),0)
        END AS predicted_survival_time,
        CASE 
            WHEN s.current_trailing_term <= 7 THEN '초단기'
            WHEN s.current_trailing_term <= 28 THEN '단기'
            WHEN s.current_trailing_term <= 60 THEN '중기'
            ELSE '장기'
        END as cycle_length,
        CASE 
            WHEN i.survival_prob <= 0.2 THEN 'High-Risk'
            WHEN i.survival_prob <= 0.5 THEN 'Medium-Risk'
            WHEN i.survival_prob <= 0.8 THEN 'Low-Risk'
            ELSE 'Safe'
        END AS risk_level
    FROM survival_base s
    LEFT JOIN individual_survival i ON s.user_id = i.user_id
    LEFT JOIN robust_bounds r ON s.user_id = r.user_id
),

new_users AS (
    SELECT 
        b.user_id,
        b.order_dt as latest_order_dt,
        date_diff(date({end_date_plus}), b.order_dt, day) as days_since_last_purchase
    FROM base_data b
    WHERE b.recency_rank = 1
    AND NOT EXISTS (
        SELECT 1 FROM purchase_intervals p 
        WHERE p.user_id = b.user_id 
        AND p.purchase_rank > 1
    )
),

new_user_stats AS (
    select *,
        case 
            when survival_prob <= 0.2 then 'High-Risk'
            when survival_prob <= 0.5 then 'Medium-Risk'
            when survival_prob <= 0.8 then 'Low-Risk'
            else 'Safe'
        end as risk_level
    from
        (SELECT 
            n.user_id,
            n.days_since_last_purchase,
            NULL as current_trailing_term,
            NULL as prev_trailing_term,
            NULL as cycle_stddev,
            NULL as modified_zscore,
            NULL as demographic_churn_rate,
            NULL as churn_flag,
            0.7 * EXP(-n.days_since_last_purchase / t.pop_mean) +
            0.3 * COALESCE(k.cumulative_survival_prob, m.min_survival_prob) as survival_prob,
            GREATEST(-t.pop_mean * LN(0.5) - n.days_since_last_purchase, 0) as predicted_survival_time,
            '신규' as cycle_length
        FROM new_users n
        CROSS JOIN (SELECT DISTINCT pop_mean FROM term_stats) t
        LEFT JOIN km_estimate k 
            ON k.time_point = FLOOR(n.days_since_last_purchase / 7) * 7
        CROSS JOIN min_survival_prob m)
)

SELECT *, date({end_date_plus}) as cur FROM new_user_stats
UNION ALL
SELECT *, date({end_date_plus}) as cur FROM final_analysis
)"""


### rfm-trg4-create-merge-byshop-0102

In [6]:
bycommerce_5 = """create or replace table ballosodeuk.ynam.rfm_table_merged_byshop as (

WITH
-------------------------------------------------------------------------------
-- 1) 쿠팡 기준: frequency >= 1
byshop_base AS (
SELECT DISTINCT
    user_id,
    platform,
    join_dt,
    cum_lifetime,
    Null as gender,
    Null as birth_year,
    total_accumulate_cash,
    recency           AS byrecency,
    frequency         AS bycommerce_f,   -- 쿠팡 구매 빈도
    monetary          AS bycommerce_m    -- 쿠팡 구매 금액
FROM ballosodeuk.ynam.rfm_table_bycommerce
WHERE monetary >= 1 -- 1/9 수정
),

-------------------------------------------------------------------------------
-- 2) 쇼핑몰 RFM 병합 (LEFT JOIN)
byshop_left_shopby AS (
SELECT
    b.user_id,
    COALESCE(b.platform, s.platform)         AS platform,
    COALESCE(b.join_dt, s.join_dt)           AS join_dt,
    COALESCE(b.cum_lifetime, s.cum_lifetime) AS cum_lifetime,
    COALESCE(s.gender, Null)            AS gender,
    COALESCE(s.birth_year, Null)    AS birth_year,
    COALESCE(b.total_accumulate_cash, s.total_accumulate_cash) AS total_accumulate_cash,
    
    b.byrecency       AS byrecency,
    b.bycommerce_f    AS bycommerce_f,
    b.bycommerce_m    AS bycommerce_m,
    
    -- 쇼핑몰(상세 RFM)
    s.recency         AS recency,      -- 쇼핑몰 recency
    s.frequency       AS commerce_f,
    s.monetary        AS commerce_m
FROM byshop_base b
LEFT JOIN ballosodeuk.ynam.rfm_table_shopby s
    ON b.user_id = s.user_id
),

-------------------------------------------------------------------------------
-- 3) 오퍼월 RFM 병합 (LEFT JOIN)
merged_rfm AS (
SELECT 
    w.user_id,
    w.platform,
    w.join_dt,
    w.cum_lifetime,
    w.gender,
    w.birth_year,
    w.total_accumulate_cash,

    -- 쇼핑몰 RFM
    w.recency     AS recency,
    w.commerce_f  AS commerce_f,
    w.commerce_m  AS commerce_m,
    
    -- 쿠팡 RFM
    w.byrecency   AS byrecency,
    w.bycommerce_f AS bycommerce_f,
    w.bycommerce_m AS bycommerce_m,

    -- noncommerce RFM
    n.recency     AS nonrecency,
    n.frequency   AS noncommerce_f,
    n.monetary    AS noncommerce_m

FROM byshop_left_shopby w
LEFT JOIN ballosodeuk.ynam.rfm_table_noncommerce n
    ON w.user_id = n.user_id
),

-------------------------------------------------------------------------------
-- 4) 쇼지, 서바이벌, 카테고리 파워 등 부가 정보
fill_shoji_properties AS (
SELECT 
    b.wk_id AS user_id,
    SUM(CASE WHEN a.accumulation_status IN ('취소로 인한 지급','지급') 
        AND DATE(a.register_dttm) <= date({end_date}) 
        THEN a.amt END) AS accumulate_shoji,
    COALESCE(
    SUM(CASE WHEN a.accumulation_status IN ('취소로 인한 지급','지급') 
        AND DATE(a.register_dttm) <= date({end_date}) 
        THEN a.amt END)
    - SUM(CASE WHEN a.accumulation_status IN ('차감') 
        AND DATE(a.register_dttm) <= date({end_date}) 
        THEN a.amt END),
    0
    ) AS current_shoji
FROM ballosodeuk.dw.fact_shopby_reward a
LEFT JOIN ballosodeuk.dw.dim_shopby_member b
    ON a.member_no = b.member_no
WHERE DATE(a.register_dttm) <= date({end_date})
GROUP BY user_id
),

fill_shopby_churn_properties AS (
SELECT *
FROM ballosodeuk.ynam.rfm_table_shopby_survive_prop
),

fill_byshop_churn_properties AS (
SELECT *
FROM ballosodeuk.ynam.rfm_table_byshop_survive_prop
),

shopby_categorypower AS (
SELECT *
FROM ballosodeuk.ynam.rfm_table_shopby_category_power
),

byshop_categorypower AS (
SELECT *
FROM ballosodeuk.ynam.rfm_table_byshop_category_power
),

-------------------------------------------------------------------------------
-- 5) 유저 프로퍼티 최종 필링
fill_userproperties AS (
SELECT 
    /* merged_rfm */ a.* EXCEPT(gender, birth_year, join_dt, platform, total_accumulate_cash)
    
    -- 기본 유저 정보
    ,CASE WHEN a.gender     IS NULL THEN b.gender     ELSE a.gender     END AS gender
    ,CASE WHEN a.birth_year IS NULL THEN b.birth_year ELSE a.birth_year END AS birth_year
    ,CASE WHEN a.join_dt    IS NULL THEN b.join_dt    ELSE a.join_dt    END AS join_dt
    ,CASE WHEN a.platform   IS NULL THEN b.platform   ELSE a.platform   END AS platform
    
    ,CASE WHEN a.total_accumulate_cash IS NULL THEN b.total_accumulate_cash
        ELSE a.total_accumulate_cash END AS total_accumulate_cash
    
    ,b.current_cash
    ,b.terms_agree_yn
    
    -- 쇼지
    ,c.accumulate_shoji
    ,c.current_shoji
    
    -- shopby churn
    ,d.days_since_last_purchase      AS last_purchase_shop
    ,d.current_trailing_term         AS current_trailing_term_shop
    ,d.prev_trailing_term            AS prev_trailing_term_shop
    ,d.cycle_stddev                  AS cycle_stddev_shop
    ,d.survival_prob                 AS suvival_prob_shop
    ,d.predicted_survival_time       AS predicted_survival_time_shop
    ,d.risk_level                    AS risk_level_shop
    ,d.cycle_length                  AS cycle_length_shop
    
    -- byshop churn
    ,e.days_since_last_purchase      AS last_purchase_byshop
    ,e.current_trailing_term         AS current_trailing_term_byshop
    ,e.prev_trailing_term           AS prev_trailing_term_byshop
    ,e.cycle_stddev                  AS cycle_stddev_byshop
    ,e.survival_prob                 AS suvival_prob_byshop
    ,e.predicted_survival_time       AS predicted_survival_time_byshop
    ,e.risk_level                    AS risk_level_byshop
    ,e.cycle_length                  AS cycle_length_byshop
    
    -- shopby 카테고리 파워
    ,f.ranking_1_1      AS ranking_1_1_sp
    ,f.ranking_1_2      AS ranking_1_2_sp
    ,f.ranking_1_3      AS ranking_1_3_sp
    ,f.power_1_1        AS power_1_1_sp
    ,f.power_1_2        AS power_1_2_sp
    ,f.power_1_3        AS power_1_3_sp
    ,f.ranking_2_1      AS ranking_2_1_sp
    ,f.ranking_2_2      AS ranking_2_2_sp
    ,f.ranking_2_3      AS ranking_2_3_sp
    ,f.ranking_2_4      AS ranking_2_4_sp
    ,f.ranking_2_5      AS ranking_2_5_sp
    ,f.ranking_2_6      AS ranking_2_6_sp
    ,f.power_2_1        AS power_2_1_sp
    ,f.power_2_2        AS power_2_2_sp
    ,f.power_2_3        AS power_2_3_sp
    ,f.power_2_4        AS power_2_4_sp
    ,f.power_2_5        AS power_2_5_sp
    ,f.power_2_6        AS power_2_6_sp
    
    -- byshop 카테고리 파워
    ,g.ranking_1_1      AS ranking_1_1_bs
    ,g.ranking_1_2      AS ranking_1_2_bs
    ,g.ranking_1_3      AS ranking_1_3_bs
    ,g.power_1_1        AS power_1_1_bs
    ,g.power_1_2        AS power_1_2_bs
    ,g.power_1_3        AS power_1_3_bs
    ,g.ranking_2_1      AS ranking_2_1_bs
    ,g.ranking_2_2      AS ranking_2_2_bs
    ,g.ranking_2_3      AS ranking_2_3_bs
    ,g.ranking_2_4      AS ranking_2_4_bs
    ,g.ranking_2_5      AS ranking_2_5_bs
    ,g.ranking_2_6      AS ranking_2_6_bs
    ,g.power_2_1        AS power_2_1_bs
    ,g.power_2_2        AS power_2_2_bs
    ,g.power_2_3        AS power_2_3_bs
    ,g.power_2_4        AS power_2_4_bs
    ,g.power_2_5        AS power_2_5_bs
    ,g.power_2_6        AS power_2_6_bs
    
FROM merged_rfm a

LEFT JOIN (
    SELECT 
    inner_a.user_id,
    inner_a.join_dt,
    inner_a.platform,
    inner_a.current_cash,
    inner_a.total_accumulate_cash,
    inner_a.terms_agree_yn,
    inner_b.gender,
    inner_b.birth_year
    FROM ballosodeuk.dw.dim_airbridge_member inner_a
    LEFT JOIN ballosodeuk.dw.dim_shopby_member inner_b
    ON inner_a.user_id = inner_b.wk_id
) b
    ON a.user_id = b.user_id

LEFT JOIN fill_shoji_properties c
    ON a.user_id = c.user_id

LEFT JOIN fill_shopby_churn_properties d
    ON a.user_id = d.user_id

LEFT JOIN fill_byshop_churn_properties e
    ON a.user_id = e.user_id

LEFT JOIN shopby_categorypower f
    ON a.user_id = f.user_id

LEFT JOIN byshop_categorypower g
    ON a.user_id = g.user_id
),

-------------------------------------------------------------------------------
-- 6) 마지막 GROUP BY + (tgt 구분, snapshot_dt 등)
source_table AS (
SELECT 
    user_id,
    MAX(platform)                       AS platform,
    MAX(join_dt)                        AS join_dt,
    MAX(cum_lifetime)                  AS cum_lifetime,
    MAX(gender)                        AS gender,
    MAX(birth_year)                    AS birth_year,
    MAX(terms_agree_yn)                AS terms_agree_yn,

    -- 재산
    MAX(COALESCE(cast(total_accumulate_cash as int64),0)) AS total_accumulate_cash,
    MAX(COALESCE(accumulate_shoji,0))      AS total_accumulate_shoji,
    MAX(COALESCE(cast(current_cash as int64),0))         AS current_cash,
    MAX(COALESCE(current_shoji,0))        AS current_shoji,

    --  유저의 쇼핑 프로퍼티
    -- --  쇼핑
    max (last_purchase_shop) as last_purchase_shop,
    max(current_trailing_term_shop) as current_trailing_term_shop, 
    max(prev_trailing_term_shop) as prev_trailing_term_shop,
    max(cycle_stddev_shop) as cycle_stddev_shop,
    max(suvival_prob_shop) as suvival_prob_shop,
    max(predicted_survival_time_shop) as predicted_survival_time_shop,
    max(risk_level_shop) as risk_level_shop,
    max(cycle_length_shop) as cycle_length_shop,

    -- --  쿠팡
    max (last_purchase_byshop) as last_purchase_byshop,
    max(current_trailing_term_byshop) as current_trailing_term_byshop, 
    max(prev_trailing_term_byshop) as prev_trailing_term_byshop,
    max(cycle_stddev_byshop) as cycle_stddev_byshop,
    max(suvival_prob_byshop) as suvival_prob_byshop,
    max(predicted_survival_time_byshop) as predicted_survival_time_byshop,
    max(risk_level_byshop) as risk_level_byshop,
    max(cycle_length_byshop) as cycle_length_byshop,

    -- 쇼핑몰 RFM
    COALESCE(MIN(recency), 99999)     AS r_shop,
    MAX(COALESCE(CAST(commerce_f AS INT64), 0)) AS f_shop,
    MAX(COALESCE(CAST(commerce_m AS INT64), 0)) AS m_shop,

    -- 쿠팡 RFM
    COALESCE(MIN(byrecency), 99999)   AS r_byshop,
    MAX(COALESCE(CAST(bycommerce_f AS INT64), 0)) AS f_byshop,
    MAX(COALESCE(CAST(bycommerce_m AS INT64), 0)) AS m_byshop,

    -- 오퍼월 RFM
    COALESCE(MIN(nonrecency), 99999) AS r_noncommerce,
    MAX(COALESCE(CAST(noncommerce_f AS INT64), 0)) AS f_noncommerce,
    MAX(COALESCE(CAST(noncommerce_m AS INT64), 0)) AS m_noncommerce,

    -- 3개 합
    COALESCE(MAX(noncommerce_m),0) + COALESCE(MAX(commerce_m),0) + COALESCE(MAX(bycommerce_m),0) AS m_total,
    COALESCE(MAX(noncommerce_f),0) + COALESCE(MAX(commerce_f),0) + COALESCE(MAX(bycommerce_f),0) AS f_total,

    -- 카테고리 파워(쇼핑몰)
    MAX(ranking_1_1_sp) AS ranking_1_1_sp,
    MAX(ranking_1_2_sp) AS ranking_1_2_sp,
    MAX(ranking_1_3_sp) AS ranking_1_3_sp,
    MAX(power_1_1_sp)   AS power_1_1_sp,
    MAX(power_1_2_sp)   AS power_1_2_sp,
    MAX(power_1_3_sp)   AS power_1_3_sp,
    MAX(ranking_2_1_sp) AS ranking_2_1_sp,
    MAX(ranking_2_2_sp) AS ranking_2_2_sp,
    MAX(ranking_2_3_sp) AS ranking_2_3_sp,
    MAX(ranking_2_4_sp) AS ranking_2_4_sp,
    MAX(ranking_2_5_sp) AS ranking_2_5_sp,
    MAX(ranking_2_6_sp) AS ranking_2_6_sp,
    MAX(power_2_1_sp)   AS power_2_1_sp,
    MAX(power_2_2_sp)   AS power_2_2_sp,
    MAX(power_2_3_sp)   AS power_2_3_sp,
    MAX(power_2_4_sp)   AS power_2_4_sp,
    MAX(power_2_5_sp)   AS power_2_5_sp,
    MAX(power_2_6_sp)   AS power_2_6_sp,

    -- 카테고리 파워(쿠팡)
    MAX(ranking_1_1_bs) AS ranking_1_1_bs,
    MAX(ranking_1_2_bs) AS ranking_1_2_bs,
    MAX(ranking_1_3_bs) AS ranking_1_3_bs,
    MAX(power_1_1_bs)   AS power_1_1_bs,
    MAX(power_1_2_bs)   AS power_1_2_bs,
    MAX(power_1_3_bs)   AS power_1_3_bs,
    MAX(ranking_2_1_bs) AS ranking_2_1_bs,
    MAX(ranking_2_2_bs) AS ranking_2_2_bs,
    MAX(ranking_2_3_bs) AS ranking_2_3_bs,
    MAX(ranking_2_4_bs) AS ranking_2_4_bs,
    MAX(ranking_2_5_bs) AS ranking_2_5_bs,
    MAX(ranking_2_6_bs) AS ranking_2_6_bs,
    MAX(power_2_1_bs)   AS power_2_1_bs,
    MAX(power_2_2_bs)   AS power_2_2_bs,
    MAX(power_2_3_bs)   AS power_2_3_bs,
    MAX(power_2_4_bs)   AS power_2_4_bs,
    MAX(power_2_5_bs)   AS power_2_5_bs,
    MAX(power_2_6_bs)   AS power_2_6_bs

FROM fill_userproperties
GROUP BY user_id
)

SELECT
CASE
    WHEN f_shop > 0 THEN "shopping"
    ELSE "non-shopping"
END AS tgt,
*,
date({end_date_plus}) AS snapshot_dt
FROM source_table
)
"""

### rfm-trg5-create-byshop-score-0102

In [7]:
bycommerce_6 = """create or replace table ballosodeuk.ynam.rfm_table_byshop_rfm_target as (
WITH r_table AS (
SELECT *
FROM ballosodeuk.ynam.rfm_table_merged_byshop
)

,rfm_scores AS (
SELECT  
    tgt,
    user_id,
    gender,
    CONCAT(
    FORMAT_DATE('%y', join_dt),
    '-',
    FORMAT_DATE('%m', join_dt)
    ) as join_group,
    CAST(FLOOR((EXTRACT(YEAR FROM date({event_date_plus})) - SAFE_CAST(birth_year AS INT64)) / 10) * 10 AS STRING) as age_group,
    platform,
    total_accumulate_cash,
    current_cash,
    terms_agree_yn,
    total_accumulate_shoji,
    current_shoji,
    r_byshop,
    f_byshop,
    m_byshop,
    current_trailing_term_byshop,
    prev_trailing_term_byshop,
    cycle_stddev_byshop,
    risk_level_byshop,
    round(suvival_prob_byshop,2) as suvival_prob_byshop,
    cycle_length_byshop,
    round(predicted_survival_time_byshop) as predicted_survival_time_byshop,

    -- Recency 점수: 낮을수록 최근 → 높은 점수 (1=낮은, 5=높은)
    NTILE(10) OVER (ORDER BY r_byshop DESC) AS recency_ntile,

    -- Frequency 점수: 높을수록 → 높은 점수 (1=낮은, 5=높은)
    -- 1/9 로직 변경 >> 1회구매자가 너무 많아 로그 변환으로 변별력 부여
    CASE 
        WHEN f_byshop = 1 THEN 1
            ELSE NTILE(9) OVER (
                PARTITION BY CASE WHEN f_byshop > 1 THEN 1 END 
                ORDER BY LN(f_byshop)) + 1
            END AS f_ntile,

    -- Monetary 점수: 높을수록 → 높은 점수 (1=낮은, 5=높은)
    NTILE(10) OVER (ORDER BY m_byshop ASC) AS m_ntile,


    -- Current Trailing Term: 작을수록 → 높은 점수 (1=긴, 5=짧은)
    CASE
        WHEN current_trailing_term_byshop is Null then 1
        ELSE
            NTILE(9) OVER (
                PARTITION BY CASE WHEN current_trailing_term_byshop is not Null then 1 END
                ORDER BY current_trailing_term_byshop DESC) + 1
        END AS term_ntile,

    -- Term Difference: 작거나 음수일수록 → 높은 점수 
    CASE
        WHEN prev_trailing_term_byshop is Null then 1
        ELSE
            NTILE(9) OVER (
                PARTITION BY CASE WHEN 
                    (current_trailing_term_byshop - prev_trailing_term_byshop) is not Null then 1 END
                ORDER BY (current_trailing_term_byshop - prev_trailing_term_byshop) DESC) + 1
        END AS term_diff_ntile,


    -- Volatility: 작을수록 → 높은 점수 
    CASE
        WHEN prev_trailing_term_byshop is Null then 1
        ELSE
            NTILE(9) OVER (
                PARTITION BY CASE WHEN current_trailing_term_byshop is not Null then 1 END
                ORDER BY cycle_stddev_byshop DESC) + 1
        END AS volatility_ntile

    ,ranking_1_1_bs as ranking_1_1
    ,ranking_1_2_bs as ranking_1_2
    ,ranking_1_3_bs as ranking_1_3
    ,power_1_1_bs as power_1_1
    ,power_1_2_bs as power_1_2
    ,power_1_3_bs as power_1_3
    ,ranking_2_1_bs as ranking_2_1
    ,ranking_2_2_bs as ranking_2_2
    ,ranking_2_3_bs as ranking_2_3
    ,ranking_2_4_bs as ranking_2_4
    ,ranking_2_5_bs as ranking_2_5
    ,ranking_2_6_bs as ranking_2_6
    ,power_2_1_bs as power_2_1
    ,power_2_2_bs as power_2_2
    ,power_2_3_bs as power_2_3
    ,power_2_4_bs as power_2_4
    ,power_2_5_bs as power_2_5
    ,power_2_6_bs as power_2_6
FROM r_table
WHERE f_byshop > 0
)

,final_score AS (
SELECT
    tgt,
    user_id,
    --  유저 세그
    gender,
    join_group,
    age_group,
    platform,
    total_accumulate_cash,
    current_cash,
    total_accumulate_shoji,
    current_shoji,
    terms_agree_yn,
    --  매출 세그
    r_byshop,
    f_byshop,
    m_byshop,

    round(m_byshop / f_byshop,2) as ar_byshop,
    current_trailing_term_byshop,
    prev_trailing_term_byshop,
    cycle_stddev_byshop,

    -- 점수 매김: NTILE이 클수록 높은 점수 (1~5)
    recency_ntile AS r_score,
    f_ntile AS f_score,
    m_ntile AS m_score,
    term_ntile AS term_score,
    term_diff_ntile AS term_diff_score,
    volatility_ntile AS volatility_score,

    -- 총점 계산 (가중치 적용 예시)
        ROUND(
        (recency_ntile * 0.22) + 
        (f_ntile * 0.427) + 
        (m_ntile * 0.35) + 
        (COALESCE(term_ntile, 1) * 0.0) + 
        (COALESCE(term_diff_ntile, 1) * 0.00) + 
        (COALESCE(volatility_ntile, 1) * 0.00), 2
        ) AS total_score,

        cycle_length_byshop,
        risk_level_byshop,
        suvival_prob_byshop,
        predicted_survival_time_byshop

,ranking_1_1
,ranking_1_2
,ranking_1_3
,power_1_1
,power_1_2
,power_1_3
,ranking_2_1
,ranking_2_2
,ranking_2_3
,ranking_2_4
,ranking_2_5
,ranking_2_6
,power_2_1
,power_2_2
,power_2_3
,power_2_4
,power_2_5
,power_2_6
,{event_date_plus} as snapshot_dt
FROM rfm_scores
)

select *,
    -- 퍼센타일 적용
    percentile_cont(total_score, 0.4) over () as cut1,
    percentile_cont(total_score, 0.7) over () as cut2,
    percentile_cont(total_score, 0.9) over () as cut3,
    -- 최신 등급 적용
    case 
        when total_score > percentile_cont(total_score, 0.9) over () then 'VIP'
        when total_score > percentile_cont(total_score, 0.7) over () then 'GOLD'
        when total_score > percentile_cont(total_score, 0.4) over () then 'SILVER'
    else 'IRON'
    end as grade
from final_score
)"""

### rfm-trg6-update-byshop-fintable

In [8]:
bycommerce_7 = """CREATE table if not Exists `ballosodeuk.ynam.rfm_byshop_history_array_table` (
user_id STRING,
-- 변경이 적은 기본 정보
gender STRING,
age_group STRING,
join_group STRING,
platform STRING,
terms_agree_yn STRING,

-- 스코어/상태 이력
score_history ARRAY<STRUCT<
  snapshot_dt DATE,  
  tgt STRING,
  total_score FLOAT64,
  cut1 FLOAT64,
  cut2 FLOAT64,
  cut3 FLOAT64,
  grade STRING,
  risk_level STRING,
  cycle_length STRING,


  r_score INT64,
  f_score INT64,
  m_score INT64,
  term_score INT64,
  term_diff_score INT64,
  volatility_score INT64,

  r_data INT64,
  f_data INT64,
  m_data INT64,
  ar_data FLOAT64,

  current_trailing_term FLOAT64,
  prev_trailing_term FLOAT64,
  term_diff FLOAT64,
  cycle_stddev FLOAT64,

  survival_prob FLOAT64,
  predicted_survival_time FLOAT64,
  total_accumulate_cash INT64,
  total_accumulate_shoji INT64,
  current_cash INT64,
  current_shoji INT64
>>,

-- 카테고리 관심도 이력
category_history ARRAY<STRUCT<
  snapshot_dt DATE,
  ranking_1_1 STRING,
  ranking_1_2 STRING,
  ranking_1_3 STRING,
  power_1_1 FLOAT64,
  power_1_2 FLOAT64,
  power_1_3 FLOAT64,
  ranking_2_1 STRING,
  ranking_2_2 STRING,
  ranking_2_3 STRING,
  ranking_2_4 STRING,
  ranking_2_5 STRING,
  ranking_2_6 STRING,
  power_2_1 FLOAT64,
  power_2_2 FLOAT64,
  power_2_3 FLOAT64,
  power_2_4 FLOAT64,
  power_2_5 FLOAT64,
  power_2_6 FLOAT64
>>
);

MERGE `ballosodeuk.ynam.rfm_byshop_history_array_table` T
USING (
  SELECT 
      user_id,
      gender,
      age_group,
      join_group,
      platform,
      terms_agree_yn,
      tgt,
      CAST(snapshot_dt as DATE) as snapshot_dt,
      CAST(total_score AS FLOAT64) as total_score,
      CAST(cut1 AS FLOAT64) as cut1,
      CAST(cut2 AS FLOAT64) as cut2,
      CAST(cut3 AS FLOAT64) as cut3,
      grade,
      risk_level_byshop as risk_level,
      cycle_length_byshop as cycle_length,
      CAST(r_score AS INT64) as r_score,
      CAST(f_score AS INT64) as f_score,
      CAST(m_score AS INT64) as m_score,
      CAST(term_score AS INT64) as term_score,
      CAST(term_diff_score AS INT64) as term_diff_score,
      CAST(volatility_score AS INT64) as volatility_score,
      CAST(r_byshop AS INT64) as r_data,
      CAST(f_byshop AS INT64) as f_data,
      CAST(m_byshop AS INT64) as m_data,
      CAST(ar_byshop AS FLOAT64) as ar_data,
      CAST(current_trailing_term_byshop AS FLOAT64) as current_trailing_term,
      CAST(prev_trailing_term_byshop AS FLOAT64) as prev_trailing_term,
      CAST(term_diff_score AS FLOAT64) as term_diff,
      CAST(cycle_stddev_byshop AS FLOAT64) as cycle_stddev,
      CAST(suvival_prob_byshop AS FLOAT64) as survival_prob,
      CAST(predicted_survival_time_byshop AS FLOAT64) as predicted_survival_time,
      CAST(total_accumulate_cash AS INT64) as total_accumulate_cash,
      CAST(total_accumulate_shoji AS INT64) as total_accumulate_shoji,
      CAST(current_cash AS INT64) as current_cash,
      CAST(current_shoji AS INT64) as current_shoji,
      ranking_1_1,
      ranking_1_2,
      ranking_1_3,
      power_1_1,
      power_1_2,
      power_1_3,
      ranking_2_1,
      ranking_2_2,
      ranking_2_3,
      ranking_2_4,
      ranking_2_5,
      ranking_2_6,
      power_2_1,
      power_2_2, 
      power_2_3,
      power_2_4,
      power_2_5,
      power_2_6
  FROM `ballosodeuk.ynam.rfm_table_byshop_rfm_target`
  where user_id is not Null
) S
ON T.user_id = S.user_id
WHEN MATCHED THEN
  UPDATE
  SET
      gender = S.gender,
      age_group = S.age_group,
      join_group = S.join_group,
      platform = S.platform,
      terms_agree_yn = S.terms_agree_yn,
      
      score_history = ARRAY_CONCAT(
      IFNULL(T.score_history, []), 
      [STRUCT(
          S.snapshot_dt,
          S.tgt,
          S.total_score,
          S.cut1,
          S.cut2,
          S.cut3,
          S.grade,
          S.risk_level,
          S.cycle_length,
          S.r_score,
          S.f_score,
          S.m_score,
          S.term_score,
          S.term_diff_score,
          S.volatility_score,
          S.r_data,
          S.f_data,
          S.m_data,
          S.ar_data,
          S.current_trailing_term,
          S.prev_trailing_term,
          S.term_diff,
          S.cycle_stddev,
          S.survival_prob,
          S.predicted_survival_time,
          S.total_accumulate_cash,
          S.total_accumulate_shoji,
          S.current_cash,
          S.current_shoji
      )]),
      
      category_history = ARRAY_CONCAT(
      IFNULL(T.category_history, []), 
      [STRUCT(
          S.snapshot_dt,
          S.ranking_1_1,
          S.ranking_1_2,
          S.ranking_1_3,
          S.power_1_1,
          S.power_1_2,
          S.power_1_3,
          S.ranking_2_1,
          S.ranking_2_2,
          S.ranking_2_3,
          S.ranking_2_4,
          S.ranking_2_5,
          S.ranking_2_6,
          S.power_2_1,
          S.power_2_2,
          S.power_2_3,
          S.power_2_4,
          S.power_2_5,
          S.power_2_6
      )])
WHEN NOT MATCHED THEN
  INSERT (
      user_id, 
      gender,
      age_group,
      join_group,
      platform,
      terms_agree_yn,
      score_history, 
      category_history
  )
  VALUES (
      S.user_id,
      S.gender,
      S.age_group,
      S.join_group,
      S.platform,
      S.terms_agree_yn,
      [STRUCT(
          S.snapshot_dt,
          S.tgt,
          S.total_score,
          S.cut1,
          S.cut2,
          S.cut3,
          S.grade,
          S.risk_level,
          S.cycle_length,
          S.r_score,
          S.f_score,
          S.m_score,
          S.term_score,
          S.term_diff_score,
          S.volatility_score,
          S.r_data,
          S.f_data,
          S.m_data,
          S.ar_data,
          S.current_trailing_term,
          S.prev_trailing_term,
          S.term_diff,
          S.cycle_stddev,
          S.survival_prob,
          S.predicted_survival_time,
          S.total_accumulate_cash,
          S.total_accumulate_shoji,
          S.current_cash,
          S.current_shoji
      )],
      [STRUCT(
          S.snapshot_dt,
          S.ranking_1_1,
          S.ranking_1_2,
          S.ranking_1_3,
          S.power_1_1,
          S.power_1_2,
          S.power_1_3,
          S.ranking_2_1,
          S.ranking_2_2,
          S.ranking_2_3,
          S.ranking_2_4,
          S.ranking_2_5,
          S.ranking_2_6,
          S.power_2_1,
          S.power_2_2,
          S.power_2_3,
          S.power_2_4,
          S.power_2_5,
          S.power_2_6
      )]
  )
"""

# 쇼핑

### rfm-trg1-create-shopby-1213

In [38]:
shopby_1 = """CREATE or Replace TABLE ballosodeuk.ynam.rfm_table_shopby AS (
WITH date_vars AS (
    SELECT DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY) as yesterday
)

,valid_orders AS (
SELECT 
    order_dt,
    a.member_no,
    b.wk_id as user_id,
    c.join_dt,
    c.total_accumulate_cash,
    c.current_cash,
    c.platform,
    b.gender,
    b.birth_year,
    product_option[SAFE_OFFSET(0)].commission_rate as commission_rate,
    # last_main_pay_amt,
    first_pay_amt, -- 1/20 수정
    product_option[SAFE_OFFSET(0)].order_status_type as order_status_type1
FROM `ballosodeuk.dw.fact_shopby_order` a
LEFT JOIN `ballosodeuk.dw.dim_shopby_member` b 
    ON a.member_no = b.member_no
LEFT JOIN `ballosodeuk.dw.dim_airbridge_member` c
    on b.wk_id = c.user_id
WHERE product_option[SAFE_OFFSET(0)].order_status_type IN (
    'DELIVERY_DONE', 'DELIVERY_PREPARE', 'DELIVERY_ING',
    'BUY_CONFIRM', 'PAY_DONE', 'EXCHANGE_DONE', 'PRODUCT_PREPARE'
) and order_dt between date("2024-10-01") and date({end_date})
),

refund_check AS (
-- 30일 이내 환불 확인
SELECT 
    v.*,
    CASE 
    WHEN r.product_option[SAFE_OFFSET(0)].order_status_type in ('CANCEL DONE', 'RETURN_DONE')
    AND DATE_DIFF(DATE(r.order_dt), v.order_dt, DAY) <= 90 
    AND r.order_dt <= date({end_date})
    THEN TRUE 
    ELSE FALSE 
    END as is_refunded
FROM valid_orders v
LEFT JOIN `ballosodeuk.dw.fact_shopby_order` r
    ON v.member_no = r.member_no
    AND r.product_option[SAFE_OFFSET(0)].order_status_type in ('CANCEL DONE', 'RETURN_DONE')
)
,grp_table as 
(SELECT 
    r.member_no,
    r.user_id,
    MAX(r.platform) as platform,
    MAX(r.join_dt) as join_dt,
    DATE_DIFF(date({end_date}), DATE(MAX(r.join_dt)), DAY) AS cum_lifetime,  -- 수정: CURRENT_DATE -> end_date
    MAX(r.total_accumulate_cash) as total_accumulate_cash,
    MAX(r.current_cash) as current_cash,

    -- recency 계산 수정
    CASE 
        WHEN MAX(CASE WHEN NOT is_refunded THEN order_dt END) IS NULL THEN NULL  -- 구매 이력 없음
        WHEN MAX(CASE WHEN NOT is_refunded THEN order_dt END) > date({end_date}) THEN NULL  -- 미래의 구매
        ELSE DATE_DIFF(date({end_date}), 
                      MAX(CASE WHEN NOT is_refunded THEN order_dt END), 
                      DAY)  -- 마지막 구매일로부터 기준일까지의 기간
    END as recency,

    -- frequency 계산 수정
    COUNT(DISTINCT CASE 
        WHEN NOT is_refunded AND order_dt <= date({end_date})  -- 기준일까지의 구매만 카운트
        THEN order_dt 
    END) as frequency,

    -- monetary 계산 수정
    SUM(CASE 
        WHEN NOT is_refunded AND order_dt <= date({end_date})  -- 기준일까지의 구매 금액만 합산
        # THEN last_main_pay_amt * (commission_rate * 0.01) 
        THEN first_pay_amt -- 1/20 수정 main_pay 의 경우, 0 발생 케이스 확인.
        ELSE 0 
    END) as monetary,

    MAX(r.gender) as gender,
    MAX(r.birth_year) as birth_year,
    AVG(r.commission_rate) as avg_commission_rate,
    COUNT(*) as total_purchases,
    COUNT(CASE WHEN is_refunded THEN 1 END) as total_refunds,
    ROUND(COUNT(CASE WHEN is_refunded THEN 1 END) * 100.0 / NULLIF(COUNT(*), 0), 2) as refund_rate,
    MAX(CASE 
        WHEN NOT is_refunded AND order_dt <= date({end_date})  -- 기준일까지의 마지막 구매일
        THEN order_dt 
    END) as last_order_dt,
    MIN(order_dt) as first_order_dt -- 1/8 추가
FROM refund_check r
GROUP BY r.member_no, r.user_id)

select *, date({end_date}) as snapshot_dt 
from grp_table

)"""


### rfm-trg2-create-shopby-categoryPower-1223

In [39]:
shopby_2 = """CREATE OR REPLACE TABLE ballosodeuk.ynam.rfm_table_shopby_category_power AS (

WITH
----------------------------------------------------------------------
-- 1) [order_counts1] : Depth1 기준 (기존 로직)
----------------------------------------------------------------------
order_counts1 AS (
SELECT 
    member_no,
    depth1_category_no,
    MAX(depth1_category_name) AS depth1_category_name,
    MAX(order_dt) AS latest_order_dt,
    COUNT(DISTINCT order_dt) AS order_count
FROM (
    SELECT 
    order_dt, member_no, category_no,
    depth1_category_no,
    REPLACE(depth1_category_name, '/', '-') AS depth1_category_name
    FROM ballosodeuk.dw.fact_shopby_order
    LEFT JOIN ballosodeuk.dw.dim_shopby_product_category b
    ON b.depth4_category_no = category_no
        OR b.depth3_category_no = category_no
    WHERE order_dt between date("2024-10-01") and date({end_date})
) a
GROUP BY member_no, depth1_category_no
),

----------------------------------------------------------------------
-- 2) [order_counts2plus] : Depth1(Depth2) 기준 (새로운 로직)
--     depth2가 NULL이면 (없음)으로 치환
----------------------------------------------------------------------
order_counts2plus AS (
SELECT
    member_no,
    -- "depth1_category_name(depth2_category_name or 없음)" 형태로 합침
    CONCAT(
    REPLACE(depth1_category_name, '/', '-'),
    '(',
    CASE 
        WHEN depth2_category_name IS NULL 
            OR TRIM(REPLACE(depth2_category_name, '/', '-')) = ''
        THEN '없음'
        ELSE REPLACE(depth2_category_name, '/', '-')
    END,
    ')'
    ) AS depth1plus,
    MAX(order_dt) AS latest_order_dt,
    COUNT(DISTINCT order_dt) AS order_count
FROM (
    SELECT
    order_dt, member_no, category_no,
    REPLACE(depth1_category_name, '/', '-') AS depth1_category_name,
    REPLACE(depth2_category_name, '/', '-') AS depth2_category_name
    FROM ballosodeuk.dw.fact_shopby_order
    LEFT JOIN ballosodeuk.dw.dim_shopby_product_category b
    ON b.depth4_category_no = category_no
        OR b.depth3_category_no = category_no
    WHERE order_dt between date("2024-10-01") and date({end_date})
) a
WHERE depth1_category_name IS NOT NULL
    AND TRIM(depth1_category_name) != ''
    AND TRIM(depth1_category_name) != 'None'
GROUP BY member_no, depth1_category_name, depth2_category_name
),

----------------------------------------------------------------------
-- [A] Depth1 파트 (Top3)
----------------------------------------------------------------------
depth1_score AS (
SELECT
    member_no,
    depth1_category_no,
    depth1_category_name,
    order_count,
    ABS(DATE_DIFF(CURRENT_DATE(), latest_order_dt, DAY)) AS days,
    SUM(order_count) OVER (PARTITION BY member_no) AS total_order_count
FROM order_counts1
),
depth1_calc AS (
SELECT
    member_no,
    depth1_category_no,
    depth1_category_name,
    0.6 * SAFE_DIVIDE(order_count, total_order_count)
    + 0.4 * EXP(-0.1 * days) AS final_score,
    SUM(order_count) OVER (PARTITION BY member_no) AS total_order_count
FROM (
    SELECT
    member_no,
    depth1_category_no,
    depth1_category_name,
    order_count,
    days
    , total_order_count
    , EXP(-0.1 * days) AS recency_weight
    FROM depth1_score
)
),
depth1_final AS (
SELECT
    member_no,
    depth1_category_no,
    depth1_category_name,
    final_score,
    SUM(final_score) OVER (PARTITION BY member_no) AS sum_score
FROM depth1_calc
),
depth1_result AS (
SELECT
    member_no,
    depth1_category_name,
    ROUND(SAFE_DIVIDE(final_score, sum_score), 2) AS percentage
FROM depth1_final
),
rank_depth_1 AS (
SELECT
    d.*,
    ROW_NUMBER() OVER (PARTITION BY member_no ORDER BY percentage DESC) AS ranking
FROM depth1_result d
),
pivot_depth_1 AS (
SELECT
    member_no,
    MAX(CASE WHEN ranking=1 THEN depth1_category_name END) AS ranking_1_1,
    MAX(CASE WHEN ranking=2 THEN depth1_category_name END) AS ranking_1_2,
    MAX(CASE WHEN ranking=3 THEN depth1_category_name END) AS ranking_1_3,

    MAX(CASE WHEN ranking=1 THEN percentage END) AS power_1_1,
    MAX(CASE WHEN ranking=2 THEN percentage END) AS power_1_2,
    MAX(CASE WHEN ranking=3 THEN percentage END) AS power_1_3
FROM rank_depth_1
WHERE ranking <= 3
GROUP BY member_no
),

----------------------------------------------------------------------
-- [B] Depth1(Depth2) 파트 (Top6)
----------------------------------------------------------------------
score_plus AS (
SELECT
    member_no,
    depth1plus,
    ABS(DATE_DIFF(CURRENT_DATE(), latest_order_dt, DAY)) AS days,
    order_count,
    SUM(order_count) OVER (PARTITION BY member_no) AS total_order_count
FROM order_counts2plus
),
-- 1) calc_plus: final_score만 계산
calc_plus AS (
SELECT
    member_no,
    depth1plus,
    -- 최종 점수: 0.6*freq + 0.4*recency
    0.6 * SAFE_DIVIDE(order_count, total_order_count)
    + 0.4 * EXP(-0.1 * days) AS final_score
FROM (
    SELECT
    member_no,
    depth1plus,
    order_count,
    total_order_count,
    days
    -- recency_weight = EXP(-0.1 * days) 계산해도 되지만, 
    -- 여기서 바로 0.4 * exp(-0.1 * days) 해도 OK
    FROM score_plus
)
),

-- 2) final_plus: calc_plus 결과에서 sum_score 구하기
final_plus AS (
SELECT
    member_no,
    depth1plus,
    final_score,
    -- user_id별 final_score 합
    SUM(final_score) OVER (PARTITION BY member_no) AS sum_score
FROM calc_plus
)

,result_plus AS (
SELECT
    member_no,
    depth1plus,
    ROUND(SAFE_DIVIDE(final_score, sum_score), 2) AS percentage
FROM final_plus
),
rank_plus AS (
SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY member_no ORDER BY percentage DESC) AS ranking
FROM result_plus
),
pivot_depth1plus AS (
SELECT
    member_no,
    MAX(CASE WHEN ranking=1 THEN depth1plus END) AS ranking_2_1,
    MAX(CASE WHEN ranking=2 THEN depth1plus END) AS ranking_2_2,
    MAX(CASE WHEN ranking=3 THEN depth1plus END) AS ranking_2_3,
    MAX(CASE WHEN ranking=4 THEN depth1plus END) AS ranking_2_4,
    MAX(CASE WHEN ranking=5 THEN depth1plus END) AS ranking_2_5,
    MAX(CASE WHEN ranking=6 THEN depth1plus END) AS ranking_2_6,

    MAX(CASE WHEN ranking=1 THEN percentage END) AS power_2_1,
    MAX(CASE WHEN ranking=2 THEN percentage END) AS power_2_2,
    MAX(CASE WHEN ranking=3 THEN percentage END) AS power_2_3,
    MAX(CASE WHEN ranking=4 THEN percentage END) AS power_2_4,
    MAX(CASE WHEN ranking=5 THEN percentage END) AS power_2_5,
    MAX(CASE WHEN ranking=6 THEN percentage END) AS power_2_6
FROM rank_plus
WHERE ranking <= 6
GROUP BY member_no
),

----------------------------------------------------------------------
-- [C] 최종 Join (Depth1 & Depth1(Depth2)) + 멤버 매핑
----------------------------------------------------------------------
final_join AS (
SELECT
    p1.member_no,
    
    /* Depth1 전용 Top3 */
    p1.ranking_1_1, p1.ranking_1_2, p1.ranking_1_3,
    p1.power_1_1,   p1.power_1_2,   p1.power_1_3,
    
    /* Depth1(Depth2) 전용 Top6 */
    p2.ranking_2_1, p2.ranking_2_2, p2.ranking_2_3,
    p2.ranking_2_4, p2.ranking_2_5, p2.ranking_2_6,
    
    p2.power_2_1,   p2.power_2_2,   p2.power_2_3,
    p2.power_2_4,   p2.power_2_5,   p2.power_2_6
FROM pivot_depth_1 p1
LEFT JOIN pivot_depth1plus p2
    ON p1.member_no = p2.member_no
)

SELECT
m.wk_id AS user_id,
f.* EXCEPT(member_no)
FROM final_join f
LEFT JOIN ballosodeuk.dw.dim_shopby_member m
ON f.member_no = m.member_no
ORDER BY user_id

);
"""


### rfm-trg3-create-shopby-churn-1231

In [40]:
shopby_3 = """create or replace table ballosodeuk.ynam.rfm_table_shopby_survive_prop as (
WITH base_data AS (
SELECT 
    b.wk_id as user_id,
    a.order_dt,
    b.gender,
    b.birth_year,
    row_number() OVER (PARTITION BY b.wk_id ORDER BY a.order_dt DESC) as recency_rank,
    row_number() OVER (PARTITION BY b.wk_id ORDER BY a.order_dt ASC) as purchase_rank
FROM (
    SELECT member_no, order_dt
    FROM ballosodeuk.dw.fact_shopby_order
    WHERE order_dt between date("2024-10-01") and date({end_date}) --  for문 사유로 추가 1/3
    GROUP BY member_no, order_dt
) a
LEFT JOIN ballosodeuk.dw.dim_shopby_member b 
    ON a.member_no = b.member_no
),

recent_purchase as (
SELECT 
    user_id,
    order_dt as latest_order_dt,
    date_diff({end_date_plus}, order_dt, day) as days_since_last_purchase
FROM base_data
WHERE recency_rank = 1
),

purchase_intervals AS (
SELECT 
    user_id,
    order_dt,
    purchase_rank,
    LEAD(order_dt) OVER (PARTITION BY user_id ORDER BY order_dt) as next_order_date,
    date_diff(
    LEAD(order_dt) OVER (PARTITION BY user_id ORDER BY order_dt),
    order_dt,
    day
    ) as days_between_orders
FROM base_data
WHERE purchase_rank <= 15
),

user_stats as (
SELECT 
    user_id,
    stddev(days_between_orders) as cycle_stddev
FROM purchase_intervals
WHERE days_between_orders is not null
GROUP BY user_id
),

current_trailing AS (
SELECT 
    user_id,
    round(avg(days_between_orders), 1) as current_trailing_term,
    count(*) as current_count
FROM purchase_intervals
WHERE days_between_orders IS NOT NULL
AND purchase_rank >= 1 
AND purchase_rank <= 3
GROUP BY user_id
),

prev_trailing AS (
SELECT 
    user_id,
    round(avg(days_between_orders), 1) as prev_trailing_term,
    count(*) as prev_count
FROM purchase_intervals
WHERE days_between_orders IS NOT NULL
AND purchase_rank >= 2
AND purchase_rank <= 4
GROUP BY user_id
),

survival_base AS (
SELECT 
    c.user_id,
    r.days_since_last_purchase,
    c.current_trailing_term,
    p.prev_trailing_term,
    s.cycle_stddev,
    ROUND(((c.current_trailing_term - p.prev_trailing_term) / 
    NULLIF(p.prev_trailing_term, 0)) * 100, 1) as cycle_change_rate,
    ROUND((s.cycle_stddev / NULLIF(c.current_trailing_term, 0)) * 100, 1) as cycle_variation_rate,
    b.gender,
    CAST(FLOOR((EXTRACT(YEAR FROM date({end_date_plus})) - SAFE_CAST(b.birth_year AS INT64)) / 10) * 10 AS STRING) as age_group
FROM current_trailing c
LEFT JOIN prev_trailing p ON c.user_id = p.user_id
LEFT JOIN user_stats s ON c.user_id = s.user_id
LEFT JOIN recent_purchase r ON c.user_id = r.user_id
LEFT JOIN base_data b ON c.user_id = b.user_id AND b.recency_rank = 1
),

term_stats AS (
SELECT 
    current_trailing_term,
    -- 로그 변환 적용
    LN(NULLIF(current_trailing_term, 0)) as log_term,
    STDDEV(LN(NULLIF(current_trailing_term, 0))) OVER () as log_stddev,
    AVG(LN(NULLIF(current_trailing_term, 0))) OVER () as log_mean,
    STDDEV(current_trailing_term) OVER () as pop_stddev,
    AVG(current_trailing_term) OVER () as pop_mean
FROM survival_base
WHERE current_trailing_term IS NOT NULL
AND current_trailing_term > 0  -- 0 이하 제외
),

-- term_stats에서 로그 변환 추가
median_stats AS ( 
SELECT 
    APPROX_QUANTILES(current_trailing_term, 2)[OFFSET(1)] as median_term,
    -- 로그 변환된 중앙값 추가
    APPROX_QUANTILES(LN(NULLIF(current_trailing_term, 0)), 2)[OFFSET(1)] as log_median_term
FROM survival_base
WHERE current_trailing_term IS NOT NULL
AND current_trailing_term > 0
),

mad_stats AS (
SELECT
    -- 기존 MAD
    APPROX_QUANTILES(
    ABS(s.current_trailing_term - m.median_term), 
    2
    )[OFFSET(1)] as mad,
    -- 로그 변환된 MAD
    APPROX_QUANTILES(
    ABS(LN(NULLIF(s.current_trailing_term, 0)) - m.log_median_term),
    2
    )[OFFSET(1)] as log_mad
FROM survival_base s
CROSS JOIN median_stats m
WHERE s.current_trailing_term IS NOT NULL
AND s.current_trailing_term > 0
),

robust_bounds AS (
SELECT
    s.user_id,
    s.current_trailing_term,
    t.pop_mean,
    -- 로그 변환된 modified z-score 계산
    0.6745 * (LN(NULLIF(s.current_trailing_term, 0)) - m.log_median_term) / NULLIF(mad.log_mad, 0) as modified_zscore,
    s.days_since_last_purchase
FROM survival_base s
CROSS JOIN (SELECT DISTINCT pop_mean FROM term_stats) t
CROSS JOIN median_stats m
CROSS JOIN mad_stats mad
WHERE s.current_trailing_term IS NOT NULL
AND s.current_trailing_term > 0
),

churn_data AS (
SELECT 
    r.user_id,
    r.current_trailing_term,
    s.age_group,
    s.gender,
    CASE
    WHEN ABS(r.modified_zscore) > 3.5 THEN
        CASE WHEN r.days_since_last_purchase > r.pop_mean * 2 THEN 1 ELSE 0 END
    ELSE
        CASE WHEN r.days_since_last_purchase > r.current_trailing_term * 2 THEN 1 ELSE 0 END
    END as churn_flag
FROM robust_bounds r
LEFT JOIN survival_base s ON r.user_id = s.user_id
),

churn_group_count AS (
SELECT 
    age_group,
    gender,
    COUNT(*) AS group_user_count
FROM churn_data
GROUP BY age_group, gender
),

average_churn_rate AS (
SELECT 
    c.age_group,
    c.gender,
    SUM(c.churn_flag) / COUNT(*) AS avg_churn_rate,  -- 수정: 단순히 이탈한 사용자 비율 계산
    AVG(c.current_trailing_term) as avg_group_term
FROM churn_data c
GROUP BY c.age_group, c.gender
)

-- time_point 보간을 위한 기준 데이터 생성
,filled_time_points AS (
    SELECT DISTINCT 
        time_point,
        gender
    FROM (
        SELECT time_point
        FROM UNNEST(GENERATE_ARRAY(
            0, 
            (SELECT MAX(FLOOR(days_since_last_purchase / 7) * 7) FROM survival_base),
            7
        )) as time_point
    ) t
    CROSS JOIN (SELECT DISTINCT gender FROM survival_base)
),

-- 생존분석
survival_base_aggregated AS (
SELECT 
    FLOOR(s.days_since_last_purchase / 7) * 7 AS time_point, --  7일단위 코호트 이탈확률 도출
    s.gender,
    COUNT(*) AS n_risk,
    SUM(c.churn_flag) AS n_events
FROM survival_base s
LEFT JOIN churn_data c ON s.user_id = c.user_id
GROUP BY 
    FLOOR(s.days_since_last_purchase / 7) * 7,
    s.gender
)

-- 누락된 time_point 보간 처리
,interpolated_survival AS (
select *
from
    (SELECT 
        f.time_point,
        f.gender,
        COALESCE(s.n_risk,
            (LAG(s.n_risk) OVER (PARTITION BY f.gender ORDER BY f.time_point) +
            LEAD(s.n_risk) OVER (PARTITION BY f.gender ORDER BY f.time_point)) / 2
        ) as n_risk,
        COALESCE(s.n_events,
            (LAG(s.n_events) OVER (PARTITION BY f.gender ORDER BY f.time_point) +
            LEAD(s.n_events) OVER (PARTITION BY f.gender ORDER BY f.time_point)) / 2
        ) as n_events
    FROM filled_time_points f
    LEFT JOIN survival_base_aggregated s 
        ON f.time_point = s.time_point 
        AND f.gender = s.gender)
    where gender is not Null
)

-- 보간된 데이터로 KM 추정
,km_estimate AS (
SELECT 
    s.time_point,
    s.gender,
    s.n_risk,
    s.n_events,
    -- 기본 생존확률: 해당 시점에서의 생존율
    ROUND(
    CASE 
        WHEN s.n_risk > 0 THEN (1 - SAFE_DIVIDE(s.n_events, s.n_risk))
        ELSE 1 
    END, 
    4) as base_survival_prob,
    
    -- 가중 생존확률: 기본 생존율에 코호트별 평균 이탈률 반영
    ROUND(
    CASE 
        WHEN s.n_risk > 0 THEN 
        (1 - SAFE_DIVIDE(s.n_events, s.n_risk)) * 
        (1 - COALESCE(a.avg_churn_rate, 0))
        ELSE 1 
    END,
    4) as weighted_survival_prob,
    
    -- 누적 생존확률: 각 시점까지의 생존확률을 누적 곱
    ROUND(
    EXP(
        SUM(LN(
        GREATEST(
            CASE 
            WHEN s.n_risk > 0 THEN (1 - SAFE_DIVIDE(s.n_events, s.n_risk))
            ELSE 1
            END,
            0.0001
        )
        )) OVER (
        PARTITION BY s.gender 
        ORDER BY s.time_point
        )
    ), 
    4) as cumulative_survival_prob
FROM interpolated_survival s
LEFT JOIN (
    SELECT 
    gender,
    AVG(avg_churn_rate) as avg_churn_rate
    FROM average_churn_rate
    GROUP BY gender
) a ON s.gender = a.gender
)

,min_survival_prob AS (
SELECT 
    gender,
    MIN(cumulative_survival_prob) as min_survival_prob
FROM km_estimate
GROUP BY gender
)

,individual_survival AS (
SELECT 
    s.user_id,
    s.days_since_last_purchase,
    s.current_trailing_term,
    c.churn_flag,
    r.modified_zscore,
    s.age_group,
    s.gender,
    acr.avg_churn_rate,
    acr.avg_group_term,
    -- 생존확률 계산 수정
    CASE 
    WHEN s.gender IS NULL THEN
        -- gender가 NULL인 경우: 코호트 가중평균 미적용
        CASE
        WHEN ABS(r.modified_zscore) > 3.5 THEN 
            EXP(-r.days_since_last_purchase / r.pop_mean)
        ELSE
            EXP(-r.days_since_last_purchase / NULLIF(s.current_trailing_term, 0))
        END
    ELSE
        -- gender가 있는 경우: 코호트 가중평균 적용
        CASE
        WHEN ABS(r.modified_zscore) > 3.5 THEN 
            0.7 * EXP(-r.days_since_last_purchase / r.pop_mean) +
            0.3 * COALESCE(k.cumulative_survival_prob, m.min_survival_prob)
        ELSE
            0.7 * EXP(-r.days_since_last_purchase / NULLIF(s.current_trailing_term, 0)) +
            0.3 * COALESCE(k.cumulative_survival_prob, m.min_survival_prob)
        END
    END as survival_prob
FROM survival_base s
LEFT JOIN churn_data c ON s.user_id = c.user_id
LEFT JOIN robust_bounds r ON s.user_id = r.user_id
LEFT JOIN average_churn_rate acr 
    ON s.age_group = acr.age_group AND s.gender = acr.gender
LEFT JOIN km_estimate k 
    ON k.gender = s.gender 
    AND k.time_point = FLOOR(s.days_since_last_purchase / 7) * 7
LEFT JOIN min_survival_prob m
    ON m.gender = s.gender
),

final_analysis AS (
SELECT 
    s.user_id,
    s.days_since_last_purchase,
    s.age_group,
    s.gender,
    s.current_trailing_term,
    s.prev_trailing_term,
    round(s.cycle_stddev) as cycle_stddev,
    i.modified_zscore,
    i.avg_churn_rate as demographic_churn_rate,
    i.churn_flag,
    round(i.survival_prob,2) as survival_prob,
    CASE 
    WHEN ABS(i.modified_zscore) > 3.5 THEN 
        GREATEST(ROUND(-r.pop_mean * LN(0.5) - s.days_since_last_purchase),0)
    ELSE
        GREATEST(ROUND(-s.current_trailing_term * LN(0.5) - s.days_since_last_purchase),0)
    END AS predicted_survival_time,
    CASE 
    WHEN s.current_trailing_term <= 7 THEN '초단기'
    WHEN s.current_trailing_term <= 28 THEN '단기'
    WHEN s.current_trailing_term <= 60 THEN '중기'
    ELSE '장기'
    END as cycle_length,
    CASE 
    WHEN i.survival_prob <= 0.2 THEN 'High-Risk'
    WHEN i.survival_prob <= 0.5 THEN 'Medium-Risk'
    WHEN i.survival_prob <= 0.8 THEN 'Low-Risk'
    ELSE 'Safe'
    END AS risk_level
FROM survival_base s
LEFT JOIN individual_survival i ON s.user_id = i.user_id
LEFT JOIN robust_bounds r ON s.user_id = r.user_id
)
/* 위 로직 엉키지 않게 신규 유저 CTE 따로 생성. 기구매자의 대푯값 적용 */
,new_users AS (
SELECT 
    b.user_id,
    b.order_dt as latest_order_dt,
    date_diff({end_date_plus}, b.order_dt, day) as days_since_last_purchase,
    b.gender,
    CAST(FLOOR((EXTRACT(YEAR FROM date({end_date_plus})) - SAFE_CAST(b.birth_year AS INT64)) / 10) * 10 AS STRING) as age_group
FROM base_data b
WHERE b.recency_rank = 1
AND NOT EXISTS (
    SELECT 1 FROM purchase_intervals p 
    WHERE p.user_id = b.user_id 
    AND p.purchase_rank > 1
)
)

,new_user_stats AS (
select *,
    case 
    when survival_prob <= 0.2 then 'High-Risk'
    when survival_prob <= 0.5 then 'Medium-Risk'
    when survival_prob <= 0.8 then 'Low-Risk'
    else 'Safe'
    end as risk_level
from
    (SELECT 
    n.user_id,
    n.days_since_last_purchase,
    n.age_group,
    n.gender,
    NULL as current_trailing_term,
    NULL as prev_trailing_term,
    NULL as cycle_stddev,
    NULL as modified_zscore,
    NULL as demographic_churn_rate,
    NULL as churn_flag,
    -- 생존 확률 계산 수정: NULL일 경우 성별별 최소값 사용
    case 
        when n.gender is not Null then
        0.7 * EXP(-n.days_since_last_purchase / t.pop_mean) +
        0.3 * COALESCE(k.cumulative_survival_prob, m.min_survival_prob) 
        else
        EXP(-n.days_since_last_purchase / t.pop_mean)
        end as survival_prob,
    GREATEST(-t.pop_mean * LN(0.5) - n.days_since_last_purchase, 0) as predicted_survival_time,

    '신규' as cycle_length
    FROM new_users n
    CROSS JOIN (SELECT DISTINCT pop_mean FROM term_stats) t
    LEFT JOIN km_estimate k 
    ON k.gender = n.gender 
    AND k.time_point = FLOOR(n.days_since_last_purchase / 7) * 7
    LEFT JOIN min_survival_prob m
    ON m.gender = n.gender)
)

SELECT *, {end_date_plus} as cur FROM new_user_stats
union all
SELECT *, {end_date_plus} as cur FROM final_analysis
)"""


### rfm-trg4-create-merge-shopby-0102

In [41]:
shopby_4 = """create or replace table ballosodeuk.ynam.rfm_table_merged_shopby as (

WITH
-------------------------------------------------------------------------------
-- 1) 쇼핑몰 기준: frequency >= 1
shopby_base AS (
SELECT DISTINCT
    user_id,
    platform,
    join_dt,
    cum_lifetime,
    gender,
    birth_year,
    total_accumulate_cash,
    recency,                     -- commerce_r
    frequency AS commerce_f,     -- 쇼핑몰 구매 빈도
    monetary  AS commerce_m,      -- 쇼핑몰 구매 금액
    first_order_dt, -- 1/8 추가
    refund_rate -- 1/8 추가

FROM ballosodeuk.ynam.rfm_table_shopby
WHERE monetary >= 1 -- 1/9 수정 (as is frequency)
),

-------------------------------------------------------------------------------
-- 2) 쿠팡 RFM 병합 (LEFT JOIN)
shopby_left_bycommerce AS (
SELECT
    sb.user_id,
    COALESCE(sb.platform, b.platform) AS platform,
    COALESCE(sb.join_dt, b.join_dt)   AS join_dt,
    COALESCE(sb.cum_lifetime, b.cum_lifetime) AS cum_lifetime,
    sb.gender,
    sb.birth_year,
    COALESCE(sb.total_accumulate_cash, b.total_accumulate_cash) AS total_accumulate_cash,
    sb.first_order_dt,
    sb.refund_rate,
    
    sb.recency  AS recency,      -- 쇼핑몰 recency
    sb.commerce_f AS commerce_f, -- 쇼핑몰 freq
    sb.commerce_m AS commerce_m, -- 쇼핑몰 monetary
    
    b.recency    AS byrecency,   -- 쿠팡 recency
    b.frequency  AS bycommerce_f,
    b.monetary   AS bycommerce_m
FROM shopby_base sb
LEFT JOIN ballosodeuk.ynam.rfm_table_bycommerce b
    ON sb.user_id = b.user_id
),

-------------------------------------------------------------------------------
-- 3) 오퍼월 RFM 병합 (LEFT JOIN)
merged_rfm AS (
SELECT 
    w.user_id,
    w.platform,
    w.join_dt,
    w.cum_lifetime,
    w.gender,
    w.birth_year,
    w.total_accumulate_cash,
    w.first_order_dt,
    w.refund_rate,

    w.recency,
    w.commerce_f,
    w.commerce_m,

    w.byrecency,
    w.bycommerce_f,
    w.bycommerce_m,

    /* noncommerce 병합 */
    n.recency   AS nonrecency,
    n.frequency AS noncommerce_f,
    n.monetary  AS noncommerce_m
    
FROM shopby_left_bycommerce w
LEFT JOIN ballosodeuk.ynam.rfm_table_noncommerce n
    ON w.user_id = n.user_id
),

-------------------------------------------------------------------------------
-- 4) 쇼지, 서바이벌, 카테고리 파워 등 부가 정보
fill_shoji_properties AS (
SELECT 
    b.wk_id AS user_id,
    SUM(CASE WHEN a.accumulation_status IN ('취소로 인한 지급','지급') 
        AND DATE(a.register_dttm) <= date({end_date}) 
        THEN a.amt END) AS accumulate_shoji,
    COALESCE(
    SUM(CASE WHEN a.accumulation_status IN ('취소로 인한 지급','지급') 
        AND DATE(a.register_dttm) <= date({end_date}) 
        THEN a.amt END)
    - SUM(CASE WHEN a.accumulation_status IN ('차감') 
        AND DATE(a.register_dttm) <= date({end_date}) 
        THEN a.amt END),
    0
    ) AS current_shoji
FROM ballosodeuk.dw.fact_shopby_reward a
LEFT JOIN ballosodeuk.dw.dim_shopby_member b
    ON a.member_no = b.member_no
WHERE DATE(a.register_dttm) <= date({end_date})
GROUP BY user_id
),

fill_shopby_churn_properties AS (
SELECT *
FROM ballosodeuk.ynam.rfm_table_shopby_survive_prop
),

fill_byshop_churn_properties AS (
SELECT *
FROM ballosodeuk.ynam.rfm_table_byshop_survive_prop
),

shopby_categorypower AS (
SELECT *
FROM ballosodeuk.ynam.rfm_table_shopby_category_power
),

byshop_categorypower AS (
SELECT *
FROM ballosodeuk.ynam.rfm_table_byshop_category_power
),

-------------------------------------------------------------------------------
-- 5) 유저 프로퍼티 최종 필링
fill_userproperties AS (
SELECT 
    /* merged_rfm */ a.* EXCEPT(gender, birth_year, join_dt, platform, total_accumulate_cash)
    
    -- 기본 유저 정보
    ,CASE WHEN a.gender     IS NULL THEN b.gender     ELSE a.gender     END AS gender
    ,CASE WHEN a.birth_year IS NULL THEN b.birth_year ELSE a.birth_year END AS birth_year
    ,CASE WHEN a.join_dt    IS NULL THEN b.join_dt    ELSE a.join_dt    END AS join_dt
    ,CASE WHEN a.platform   IS NULL THEN b.platform   ELSE a.platform   END AS platform
    ,CASE WHEN a.total_accumulate_cash IS NULL THEN b.total_accumulate_cash 
        ELSE a.total_accumulate_cash END AS total_accumulate_cash
    
    ,b.current_cash
    ,b.terms_agree_yn
    
    -- 쇼지
    ,c.accumulate_shoji
    ,c.current_shoji
    
    -- shopby churn
    ,d.days_since_last_purchase      AS last_purchase_shop
    ,d.current_trailing_term         AS current_trailing_term_shop
    ,d.prev_trailing_term            AS prev_trailing_term_shop
    ,d.cycle_stddev                  AS cycle_stddev_shop
    ,d.survival_prob                 AS suvival_prob_shop
    ,d.predicted_survival_time       AS predicted_survival_time_shop
    ,d.risk_level                    AS risk_level_shop
    ,d.cycle_length                  AS cycle_length_shop
    
    -- byshop churn
    ,e.days_since_last_purchase      AS last_purchase_byshop
    ,e.current_trailing_term         AS current_trailing_term_byshop
    ,e.prev_trailing_term           AS prev_trailing_term_byshop
    ,e.cycle_stddev                  AS cycle_stddev_byshop
    ,e.survival_prob                 AS suvival_prob_byshop
    ,e.predicted_survival_time       AS predicted_survival_time_byshop
    ,e.risk_level                    AS risk_level_byshop
    ,e.cycle_length                  AS cycle_length_byshop
    
    -- shopby 카테고리 파워
    ,f.ranking_1_1      AS ranking_1_1_sp
    ,f.ranking_1_2      AS ranking_1_2_sp
    ,f.ranking_1_3      AS ranking_1_3_sp
    ,f.power_1_1        AS power_1_1_sp
    ,f.power_1_2        AS power_1_2_sp
    ,f.power_1_3        AS power_1_3_sp
    ,f.ranking_2_1      AS ranking_2_1_sp
    ,f.ranking_2_2      AS ranking_2_2_sp
    ,f.ranking_2_3      AS ranking_2_3_sp
    ,f.ranking_2_4      AS ranking_2_4_sp
    ,f.ranking_2_5      AS ranking_2_5_sp
    ,f.ranking_2_6      AS ranking_2_6_sp
    ,f.power_2_1        AS power_2_1_sp
    ,f.power_2_2        AS power_2_2_sp
    ,f.power_2_3        AS power_2_3_sp
    ,f.power_2_4        AS power_2_4_sp
    ,f.power_2_5        AS power_2_5_sp
    ,f.power_2_6        AS power_2_6_sp
    
    -- byshop 카테고리 파워
    ,g.ranking_1_1      AS ranking_1_1_bs
    ,g.ranking_1_2      AS ranking_1_2_bs
    ,g.ranking_1_3      AS ranking_1_3_bs
    ,g.power_1_1        AS power_1_1_bs
    ,g.power_1_2        AS power_1_2_bs
    ,g.power_1_3        AS power_1_3_bs
    ,g.ranking_2_1      AS ranking_2_1_bs
    ,g.ranking_2_2      AS ranking_2_2_bs
    ,g.ranking_2_3      AS ranking_2_3_bs
    ,g.ranking_2_4      AS ranking_2_4_bs
    ,g.ranking_2_5      AS ranking_2_5_bs
    ,g.ranking_2_6      AS ranking_2_6_bs
    ,g.power_2_1        AS power_2_1_bs
    ,g.power_2_2        AS power_2_2_bs
    ,g.power_2_3        AS power_2_3_bs
    ,g.power_2_4        AS power_2_4_bs
    ,g.power_2_5        AS power_2_5_bs
    ,g.power_2_6        AS power_2_6_bs
    
FROM merged_rfm a

LEFT JOIN (
    SELECT 
    inner_a.user_id,
    inner_a.join_dt,
    inner_a.platform,
    inner_a.current_cash, 
    inner_a.total_accumulate_cash, 
    inner_a.terms_agree_yn,
    inner_b.gender,
    inner_b.birth_year
    FROM ballosodeuk.dw.dim_airbridge_member inner_a
    LEFT JOIN ballosodeuk.dw.dim_shopby_member inner_b
    ON inner_a.user_id = inner_b.wk_id
) b
    ON a.user_id = b.user_id

LEFT JOIN fill_shoji_properties c
    ON a.user_id = c.user_id

LEFT JOIN fill_shopby_churn_properties d
    ON a.user_id = d.user_id

LEFT JOIN fill_byshop_churn_properties e
    ON a.user_id = e.user_id

LEFT JOIN shopby_categorypower f
    ON a.user_id = f.user_id

LEFT JOIN byshop_categorypower g
    ON a.user_id = g.user_id
),

-------------------------------------------------------------------------------
-- 6) 마지막 그룹바이 + 필요한 로직
source_table AS (
SELECT 
    user_id,
    MAX(platform)                 AS platform,
    MAX(join_dt)                  AS join_dt,
    MAX(cum_lifetime)            AS cum_lifetime,
    MAX(gender)                  AS gender,
    MAX(birth_year)              AS birth_year,
    MAX(terms_agree_yn)          AS terms_agree_yn,
    MAX(first_order_dt)               AS first_order_dt,
    MAX(refund_rate)               AS refund_rate,

    -- 재산
    MAX(COALESCE(cast (total_accumulate_cash as int64) ,0))    AS total_accumulate_cash,
    MAX(COALESCE(accumulate_shoji,0))         AS total_accumulate_shoji,
    MAX(COALESCE(cast (current_cash as int64),0))            AS current_cash,
    MAX(COALESCE(current_shoji,0))           AS current_shoji,

    --  유저의 쇼핑 프로퍼티
    -- --  쇼핑
    max (last_purchase_shop) as last_purchase_shop,
    max(current_trailing_term_shop) as current_trailing_term_shop, 
    max(prev_trailing_term_shop) as prev_trailing_term_shop,
    max(cycle_stddev_shop) as cycle_stddev_shop,
    max(suvival_prob_shop) as suvival_prob_shop,
    max(predicted_survival_time_shop) as predicted_survival_time_shop,
    max(risk_level_shop) as risk_level_shop,
    max(cycle_length_shop) as cycle_length_shop,

    -- --  쿠팡
    max (last_purchase_byshop) as last_purchase_byshop,
    max(current_trailing_term_byshop) as current_trailing_term_byshop, 
    max(prev_trailing_term_byshop) as prev_trailing_term_byshop,
    max(cycle_stddev_byshop) as cycle_stddev_byshop,
    max(suvival_prob_byshop) as suvival_prob_byshop,
    max(predicted_survival_time_byshop) as predicted_survival_time_byshop,
    max(risk_level_byshop) as risk_level_byshop,
    max(cycle_length_byshop) as cycle_length_byshop,

    -- 쇼핑몰 RFM
    COALESCE(MIN(recency), 99999) AS r_shop,
    MAX(COALESCE(CAST(commerce_f AS INT64), 0)) AS f_shop,
    MAX(COALESCE(CAST(commerce_m AS INT64), 0)) AS m_shop,

    -- 쿠팡 RFM
    COALESCE(MIN(byrecency), 99999)    AS r_byshop,
    MAX(COALESCE(CAST(bycommerce_f AS INT64), 0)) AS f_byshop,
    MAX(COALESCE(CAST(bycommerce_m AS INT64), 0)) AS m_byshop,

    -- 오퍼월 RFM
    COALESCE(MIN(nonrecency), 99999)   AS r_noncommerce,
    MAX(COALESCE(CAST(noncommerce_f AS INT64), 0)) AS f_noncommerce,
    MAX(COALESCE(CAST(noncommerce_m AS INT64), 0)) AS m_noncommerce,

    -- 3개 합
    COALESCE(MAX(noncommerce_m),0) + COALESCE(MAX(commerce_m),0) + COALESCE(MAX(bycommerce_m),0) AS m_total,
    COALESCE(MAX(noncommerce_f),0) + COALESCE(MAX(commerce_f),0) + COALESCE(MAX(bycommerce_f),0) AS f_total,

    -- 카테고리 파워(쇼핑몰)
    MAX(ranking_1_1_sp) AS ranking_1_1_sp,
    MAX(ranking_1_2_sp) AS ranking_1_2_sp,
    MAX(ranking_1_3_sp) AS ranking_1_3_sp,
    MAX(power_1_1_sp)   AS power_1_1_sp,
    MAX(power_1_2_sp)   AS power_1_2_sp,
    MAX(power_1_3_sp)   AS power_1_3_sp,
    MAX(ranking_2_1_sp) AS ranking_2_1_sp,
    MAX(ranking_2_2_sp) AS ranking_2_2_sp,
    MAX(ranking_2_3_sp) AS ranking_2_3_sp,
    MAX(ranking_2_4_sp) AS ranking_2_4_sp,
    MAX(ranking_2_5_sp) AS ranking_2_5_sp,
    MAX(ranking_2_6_sp) AS ranking_2_6_sp,
    MAX(power_2_1_sp)   AS power_2_1_sp,
    MAX(power_2_2_sp)   AS power_2_2_sp,
    MAX(power_2_3_sp)   AS power_2_3_sp,
    MAX(power_2_4_sp)   AS power_2_4_sp,
    MAX(power_2_5_sp)   AS power_2_5_sp,
    MAX(power_2_6_sp)   AS power_2_6_sp,

    -- 카테고리 파워(쿠팡)
    MAX(ranking_1_1_bs) AS ranking_1_1_bs,
    MAX(ranking_1_2_bs) AS ranking_1_2_bs,
    MAX(ranking_1_3_bs) AS ranking_1_3_bs,
    MAX(power_1_1_bs)   AS power_1_1_bs,
    MAX(power_1_2_bs)   AS power_1_2_bs,
    MAX(power_1_3_bs)   AS power_1_3_bs,
    MAX(ranking_2_1_bs) AS ranking_2_1_bs,
    MAX(ranking_2_2_bs) AS ranking_2_2_bs,
    MAX(ranking_2_3_bs) AS ranking_2_3_bs,
    MAX(ranking_2_4_bs) AS ranking_2_4_bs,
    MAX(ranking_2_5_bs) AS ranking_2_5_bs,
    MAX(ranking_2_6_bs) AS ranking_2_6_bs,
    MAX(power_2_1_bs)   AS power_2_1_bs,
    MAX(power_2_2_bs)   AS power_2_2_bs,
    MAX(power_2_3_bs)   AS power_2_3_bs,
    MAX(power_2_4_bs)   AS power_2_4_bs,
    MAX(power_2_5_bs)   AS power_2_5_bs,
    MAX(power_2_6_bs)   AS power_2_6_bs

FROM fill_userproperties
GROUP BY user_id
)

SELECT 
CASE 
    WHEN f_shop > 0 THEN "shopping" 
    ELSE "non-shopping" 
END AS tgt,
*,
{end_date} AS snapshot_dt
FROM source_table
)
"""


### rfm-trg5-create-shopby-score-0102

In [42]:
shopby_5 = """create or replace table ballosodeuk.ynam.rfm_table_shopby_rfm_target as (
WITH r_table AS (
SELECT *
FROM ballosodeuk.ynam.rfm_table_merged_shopby
),

rfm_scores AS (
SELECT 
    tgt,
    user_id,
    first_order_dt,
    refund_rate,
    gender,
    CONCAT(
    FORMAT_DATE('%y', join_dt),
    '-',
    FORMAT_DATE('%m', join_dt)
    ) as join_group,
    CAST(FLOOR((EXTRACT(YEAR FROM date({event_date_plus})) - SAFE_CAST(birth_year AS INT64)) / 10) * 10 AS STRING) as age_group,
    platform,
    total_accumulate_cash,
    current_cash,
    terms_agree_yn,
    total_accumulate_shoji,
    current_shoji,
    r_shop,
    f_shop,
    m_shop,
    current_trailing_term_shop,
    prev_trailing_term_shop,
    cycle_stddev_shop,
    risk_level_shop,
    round(suvival_prob_shop,2) as suvival_prob_shop,
    cycle_length_shop,
    round(predicted_survival_time_shop) as predicted_survival_time_shop,

    -- Recency 점수: 낮을수록 최근 → 높은 점수 (1=낮은, 5=높은)
    NTILE(10) OVER (ORDER BY r_shop DESC) AS recency_ntile,

    -- Frequency 점수: 높을수록 → 높은 점수 (1=낮은, 5=높은)
    -- 1/9 로직 변경 >> 1회구매자가 너무 많아 로그 변환으로 변별력 부여
    CASE 
        WHEN f_shop = 1 THEN 1
        ELSE NTILE(9) OVER (
            PARTITION BY CASE WHEN f_shop > 1 THEN 1 END 
            ORDER BY LN(f_shop)) + 1
        END AS f_ntile,

    -- Monetary 점수: 높을수록 → 높은 점수 (1=낮은, 5=높은)
    NTILE(10) OVER (ORDER BY m_shop ASC) AS m_ntile,

    -- Current Trailing Term: 작을수록 → 높은 점수 (1=긴, 5=짧은)
    CASE
        WHEN current_trailing_term_shop is Null then 1
        ELSE
            NTILE(9) OVER (
                PARTITION BY CASE WHEN current_trailing_term_shop is not Null then 1 END
                ORDER BY current_trailing_term_shop DESC) + 1
        END AS term_ntile,

    -- Term Difference: 작거나 음수일수록 → 높은 점수 
    CASE
        WHEN prev_trailing_term_shop is Null then 1
        ELSE
            NTILE(9) OVER (
                PARTITION BY CASE WHEN 
                    (current_trailing_term_shop - prev_trailing_term_shop) is not Null then 1 END
                ORDER BY (current_trailing_term_shop - prev_trailing_term_shop) DESC) + 1
        END AS term_diff_ntile,

    -- Volatility: 작을수록 → 높은 점수 
    CASE
        WHEN prev_trailing_term_shop is Null then 1
        ELSE
            NTILE(9) OVER (
                PARTITION BY CASE WHEN current_trailing_term_shop is not Null then 1 END
                ORDER BY cycle_stddev_shop DESC) + 1
        END AS volatility_ntile

    ,ranking_1_1_sp as ranking_1_1
    ,ranking_1_2_sp as ranking_1_2
    ,ranking_1_3_sp as ranking_1_3
    ,power_1_1_sp as power_1_1
    ,power_1_2_sp as power_1_2
    ,power_1_3_sp as power_1_3
    ,ranking_2_1_sp as ranking_2_1
    ,ranking_2_2_sp as ranking_2_2
    ,ranking_2_3_sp as ranking_2_3
    ,ranking_2_3_sp as ranking_2_4
    ,ranking_2_3_sp as ranking_2_5
    ,ranking_2_3_sp as ranking_2_6
    ,power_2_1_sp as power_2_1
    ,power_2_2_sp as power_2_2
    ,power_2_3_sp as power_2_3
    ,power_2_3_sp as power_2_4
    ,power_2_3_sp as power_2_5
    ,power_2_3_sp as power_2_6
FROM r_table
WHERE f_shop > 0
)

,final_score AS (

    SELECT
        tgt,
        user_id,
        --  유저 세그
        gender,
        join_group,
        age_group,
        platform,
        total_accumulate_cash,
        current_cash,
        total_accumulate_shoji,
        current_shoji,
        first_order_dt,
        terms_agree_yn,
        --  매출 세그
        r_shop,
        f_shop,
        m_shop,

        round(m_shop / f_shop,2) as ar_shop,
        current_trailing_term_shop,
        prev_trailing_term_shop,
        current_trailing_term_shop - prev_trailing_term_shop as term_diff_shop,
        cycle_stddev_shop,
        refund_rate,

        -- 점수 매김: NTILE이 클수록 높은 점수 (1~5)
        recency_ntile AS r_score,
        f_ntile AS f_score,
        m_ntile AS m_score,
        term_ntile AS term_score,
        term_diff_ntile AS term_diff_score,
        volatility_ntile AS volatility_score, 

        -- 총점 계산 (가중치 적용 예시)
        ROUND(
        (recency_ntile * 0.22) + 
        (f_ntile * 0.427) + 
        (m_ntile * 0.35) + 
        (COALESCE(term_ntile, 1) * 0.0) + 
        (COALESCE(term_diff_ntile, 1) * 0.00) + 
        (COALESCE(volatility_ntile, 1) * 0.00), 2
        ) AS total_score,

        cycle_length_shop,
        risk_level_shop,
        suvival_prob_shop,
        predicted_survival_time_shop
    ,ranking_1_1
    ,ranking_1_2
    ,ranking_1_3
    ,power_1_1
    ,power_1_2
    ,power_1_3
    ,ranking_2_1
    ,ranking_2_2
    ,ranking_2_3
    ,ranking_2_4
    ,ranking_2_5
    ,ranking_2_6
    ,power_2_1
    ,power_2_2
    ,power_2_3
    ,power_2_4
    ,power_2_5
    ,power_2_6
    ,{event_date_plus} as snapshot_dt
    FROM rfm_scores
) 

select *,
    -- 퍼센타일 적용
    percentile_cont(total_score, 0.4) over () as cut1,
    percentile_cont(total_score, 0.7) over () as cut2,
    percentile_cont(total_score, 0.9) over () as cut3,
    -- 최신 등급 적용
    case 
        when total_score > percentile_cont(total_score, 0.9) over () then 'VIP'
        when total_score > percentile_cont(total_score, 0.7) over () then 'GOLD'
        when total_score > percentile_cont(total_score, 0.4) over () then 'SILVER'
    else 'IRON'
    end as grade
from final_score
)"""


### rfm-trg6-update-shopby-fintable

In [49]:
shopby_6 = """CREATE table if not Exists `ballosodeuk.ynam.rfm_shopby_history_array_table` (
user_id STRING,
-- 변경이 적은 기본 정보
gender STRING,
age_group STRING,
join_group STRING,
platform STRING,
terms_agree_yn STRING,
first_order_dt DATE,
-- 스코어/상태 이력
score_history ARRAY<STRUCT<
    snapshot_dt DATE,
    total_score FLOAT64,
    cut1 FLOAT64,
    cut2 FLOAT64,
    cut3 FLOAT64,
    grade STRING,
    risk_level STRING,
    
    cycle_length STRING,
    r_score INT64,
    f_score INT64,
    m_score INT64,
    term_score INT64,
    term_diff_score INT64,
    volatility_score INT64,
    r_data INT64,
    f_data INT64,
    m_data INT64,
    ar_data FLOAT64,
    current_trailing_term FLOAT64,
    prev_trailing_term FLOAT64,
    term_diff FLOAT64,
    cycle_stddev FLOAT64,
    refund_rate FLOAT64,
    survival_prob FLOAT64,
    predicted_survival_time FLOAT64,
    total_accumulate_cash INT64,
    total_accumulate_shoji INT64,
    current_cash INT64,
    current_shoji INT64
>>,

-- 카테고리 관심도 이력
category_history ARRAY<STRUCT<
    snapshot_dt DATE,
    ranking_1_1 STRING,
    ranking_1_2 STRING,
    ranking_1_3 STRING,
    power_1_1 FLOAT64,
    power_1_2 FLOAT64,
    power_1_3 FLOAT64,
    ranking_2_1 STRING,
    ranking_2_2 STRING,
    ranking_2_3 STRING,
    ranking_2_4 STRING,
    ranking_2_5 STRING,
    ranking_2_6 STRING,
    power_2_1 FLOAT64,
    power_2_2 FLOAT64,
    power_2_3 FLOAT64,
    power_2_4 FLOAT64,
    power_2_5 FLOAT64,
    power_2_6 FLOAT64
>>
);

MERGE `ballosodeuk.ynam.rfm_shopby_history_array_table` T
USING (
    SELECT 
        user_id,
        gender,
        age_group,
        join_group,
        platform,
        terms_agree_yn,
        CAST(total_score AS FLOAT64) as total_score, 
        first_order_dt,
        refund_rate,
        r_score,
        f_score,
        m_score,
        cut1,
        cut2,
        cut3,
        grade,
        risk_level_shop as risk_level,
        current_trailing_term_shop as current_trailing_term,
        prev_trailing_term_shop as prev_trailing_term,
        term_diff_shop as term_diff,
        cycle_stddev_shop as cycle_stddev,
        ar_shop as ar_data,

        term_score,
        term_diff_score,
        volatility_score,
        cycle_length_shop AS cycle_length,
        r_shop AS r_data,
        f_shop AS f_data,
        m_shop AS m_data,
        suvival_prob_shop AS survival_prob,
        predicted_survival_time_shop AS predicted_survival_time,
        total_accumulate_cash,
        total_accumulate_shoji,
        current_cash,
        current_shoji,
        ranking_1_1,
        ranking_1_2,
        ranking_1_3,
        power_1_1,
        power_1_2,
        power_1_3,
        ranking_2_1,
        ranking_2_2,
        ranking_2_3,
        ranking_2_4,
        ranking_2_5,
        ranking_2_6,
        power_2_1,
        power_2_2, 
        power_2_3,
        power_2_4,
        power_2_5,
        power_2_6,
        CAST(snapshot_dt as DATE) as snapshot_dt
    FROM `ballosodeuk.ynam.rfm_table_shopby_rfm_target`
    where user_id is not Null
) S
ON T.user_id = S.user_id
WHEN MATCHED THEN
    UPDATE
    SET
        gender = S.gender,
        age_group = S.age_group,
        join_group = S.join_group,
        platform = S.platform,
        terms_agree_yn = S.terms_agree_yn,
        first_order_dt = S.first_order_dt,

        score_history = ARRAY_CONCAT(
        T.score_history, 
        [STRUCT(
            S.snapshot_dt as snapshot_dt,
            S.total_score as total_score,
            S.cut1,
            S.cut2,
            S.cut3,
            S.grade,
            S.risk_level as risk_level,

            S.cycle_length as cycle_length,
            S.r_score as r_score,
            S.f_score as f_score,
            S.m_score as m_score,
            S.term_score as term_score,
            S.term_diff_score as term_diff_score,
            S.volatility_score as volatility_score,
            S.r_data as r_data,
            S.f_data as f_data,
            S.m_data as m_data,
            S.ar_data as ar_data,
            S.current_trailing_term as current_trailing_term,
            S.prev_trailing_term as prev_trailing_term,
            S.term_diff as term_diff,
            S.cycle_stddev as cycle_stddev,
            S.refund_rate as refund_rate,
            S.survival_prob as survival_prob,
            S.predicted_survival_time as predicted_survival_time,
            S.total_accumulate_cash as total_accumulate_cash,
            S.total_accumulate_shoji as total_accumulate_shoji,
            S.current_cash as current_cash,
            S.current_shoji as current_shoji
        )]
        ),
        
        category_history = ARRAY_CONCAT(
        T.category_history, 
        [STRUCT(
            S.snapshot_dt as snapshot_dt,
            S.ranking_1_1 as ranking_1_1,
            S.ranking_1_2 as ranking_1_2,
            S.ranking_1_3 as ranking_1_3,
            S.power_1_1 as power_1_1,
            S.power_1_2 as power_1_2,
            S.power_1_3 as power_1_3,
            S.ranking_2_1 as ranking_2_1,
            S.ranking_2_2 as ranking_2_2,
            S.ranking_2_3 as ranking_2_3,
            S.ranking_2_4 as ranking_2_4,
            S.ranking_2_5 as ranking_2_5,
            S.ranking_2_6 as ranking_2_6,
            S.power_2_1 as power_2_1,
            S.power_2_2 as power_2_2,
            S.power_2_3 as power_2_3,
            S.power_2_4 as power_2_4,
            S.power_2_5 as power_2_5,
            S.power_2_6 as power_2_6
        )]
        )
WHEN NOT MATCHED THEN
    INSERT (
        user_id, 
        gender,
        age_group,
        join_group,
        platform,
        terms_agree_yn,
        first_order_dt,
        score_history, 
        category_history
    )
    VALUES (
        S.user_id,
        S.gender,
        S.age_group,
        S.join_group,
        S.platform,
        S.terms_agree_yn,
        first_order_dt,
        [STRUCT(
            S.snapshot_dt as snapshot_dt,
            S.total_score as total_score,
            S.cut1,
            S.cut2,
            S.cut3,
            S.grade,    
            S.risk_level as risk_level,

            S.cycle_length as cycle_length,
            S.r_score as r_score,
            S.f_score as f_score,
            S.m_score as m_score,
            S.term_score as term_score,
            S.term_diff_score as term_diff_score,
            S.volatility_score as volatility_score,
            S.r_data as r_data,
            S.f_data as f_data,
            S.m_data as m_data,
            S.ar_data as ar_data,
            S.current_trailing_term as current_trailing_term,
            S.prev_trailing_term as prev_trailing_term,
            S.term_diff as term_diff,
            S.cycle_stddev as cycle_stddev,
            S.refund_rate as refund_rate,
            S.survival_prob as survival_prob,
            S.predicted_survival_time as predicted_survival_time,
            S.total_accumulate_cash as total_accumulate_cash,
            S.total_accumulate_shoji as total_accumulate_shoji,
            S.current_cash as current_cash,
            S.current_shoji as current_shoji
        )],
        [STRUCT(
            S.snapshot_dt as snapshot_dt,
            S.ranking_1_1 as ranking_1_1,
            S.ranking_1_2 as ranking_1_2,
            S.ranking_1_3 as ranking_1_3,
            S.power_1_1 as power_1_1,
            S.power_1_2 as power_1_2,
            S.power_1_3 as power_1_3,
            S.ranking_2_1 as ranking_2_1,
            S.ranking_2_2 as ranking_2_2,
            S.ranking_2_3 as ranking_2_3,
            S.ranking_2_4 as ranking_2_4,
            S.ranking_2_5 as ranking_2_5,
            S.ranking_2_6 as ranking_2_6,
            S.power_2_1 as power_2_1,
            S.power_2_2 as power_2_2,
            S.power_2_3 as power_2_3,
            S.power_2_4 as power_2_4,
            S.power_2_5 as power_2_5,
            S.power_2_6 as power_2_6
        )]
    )
""" 

# 실행

In [ ]:
# byshop_queries = [bycommerce_1, bycommerce_2, bycommerce_3, bycommerce_4, bycommerce_5, bycommerce_6, bycommerce_7]
# shopby_queries = [shopby_1, shopby_2, shopby_3, shopby_4, shopby_5, shopby_6]

In [8]:
folders = os.listdir('sql')
byshop_queries = []
shopby_queries = []

query_list = {
    'byshop' : byshop_queries,
    'shopby' : shopby_queries
}


for i, fd in enumerate(folders):
    files = os.listdir(f'sql/{folders[i]}')
    files = [f for f in files if f.endswith('.sql')]
    
    for file in files:
        with open(f'sql/{folders[i]}/{file}', 'r') as f:
            query =f.read()
            query_list[fd].append(query)



In [11]:
from datetime import datetime, date, timedelta
from google.cloud import bigquery
import time

def main_shopby(queries,historical_start_date,search_start_date):
    bq = bigquery.Client()
    
    
    # A: 데이터 셋의 시작 날짜 (고정)
    DATA_START_DATE = historical_start_date
    
    # B: 조회하려는 시작 날짜
    QUERY_START_DATE = search_start_date
    
    # C: 조회하려는 마감 날짜 (어제)
    QUERY_END_DATE = date.today() - timedelta(days=1)
    
    # 유효성 검사
    if QUERY_START_DATE < DATA_START_DATE:
        raise ValueError(f"조회 시작일({QUERY_START_DATE})은 데이터 시작일({DATA_START_DATE}) 이후여야 합니다.")
    if QUERY_END_DATE < QUERY_START_DATE:
        raise ValueError(f"조회 종료일({QUERY_END_DATE})은 조회 시작일({QUERY_START_DATE}) 이후여야 합니다.")
    
    # 날짜를 순차적으로 실행
    current_date = QUERY_START_DATE
    while current_date <= QUERY_END_DATE:
        cur_date = current_date.strftime('%Y-%m-%d')
        cur_date_plus = (current_date + timedelta(days=1)).strftime('%Y-%m-%d')
        
        print(f"\n=== {DATA_START_DATE.strftime('%Y-%m-%d')} 부터 {cur_date} 까지의 누적 데이터 처리 ===")
        print(f"기준일: {cur_date}, 다음날: {cur_date_plus}")

        # 각 쿼리를 개별적으로 실행
        for i, query in enumerate(queries, start=1):
            print(f"쿼리 {i} 실행 중...")
            try:
                formatted_query = query.format(
                    end_date=f"'{cur_date}'",
                    end_date_plus=f"'{cur_date_plus}'",
                    event_date_plus=f"'{cur_date}'"
                )
                
                job = bq.query(formatted_query)
                result = job.result()
                
                print(f"[쿼리 {i}] 성공")
                
            except Exception as e:
                print(f"[쿼리 {i}] 오류:", str(e))
                continue

        current_date += timedelta(days=1)

# query_list = {
#     'byshop' : byshop_queries,
#     'shopby' : shopby_queries
# }
tgt_query = "shopby"
search_start_date = date(2025, 2, 1) # 데이터 조회 날짜 기준.

if __name__ == "__main__":
    if tgt_query == "byshop":
        historical_start_date = date(2024, 3, 1)
    elif tgt_query == "shopby":
        historical_start_date = date(2024, 10, 1)
    else:
        historical_start_date = None
    total_start_time = time.time()
    
    main_shopby(query_list[tgt_query],historical_start_date,search_start_date)

    total_end_time = time.time()
    total_duration = total_end_time - total_start_time
    print(f"\n=== 전체 작업 완료 (총 소요시간: {total_duration:.2f}초) ===")



=== 2024-10-01 부터 2025-02-01 까지의 누적 데이터 처리 ===
기준일: 2025-02-01, 다음날: 2025-02-02
쿼리 1 실행 중...
[쿼리 1] 성공
쿼리 2 실행 중...
[쿼리 2] 성공
쿼리 3 실행 중...
[쿼리 3] 성공
쿼리 4 실행 중...
[쿼리 4] 성공
쿼리 5 실행 중...
[쿼리 5] 성공
쿼리 6 실행 중...
[쿼리 6] 성공

=== 2024-10-01 부터 2025-02-02 까지의 누적 데이터 처리 ===
기준일: 2025-02-02, 다음날: 2025-02-03
쿼리 1 실행 중...
[쿼리 1] 성공
쿼리 2 실행 중...
[쿼리 2] 성공
쿼리 3 실행 중...
[쿼리 3] 성공
쿼리 4 실행 중...
[쿼리 4] 성공
쿼리 5 실행 중...
[쿼리 5] 성공
쿼리 6 실행 중...
[쿼리 6] 성공

=== 전체 작업 완료 (총 소요시간: 73.15초) ===
